# 라이브러리 Install

In [13]:
%pip install --upgrade pip

%pip install qdrant_client rank_bm25 sentence_transformers langchain_text_splitters

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


# 라이브러리 Import

In [4]:
from collections import defaultdict
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, SparseVectorParams, PointStruct, SparseVector
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
from langchain_text_splitters import RecursiveCharacterTextSplitter

import numpy as np

print("✅ 라이브러리 Import 완료")

✅ 라이브러리 Import 완료


# 임베딩 모델, 벡터 DB 초기화

  📊 사용 가능한 로컬 임베딩 모델

  | 모델                                    | 차원  | 속도     | 품질      | 용도                    |
  |---------------------------------------|-----|--------|---------|-----------------------|
  | all-MiniLM-L6-v2                      | 384 | ⚡⚡⚡ 빠름 | ⭐⭐⭐ 좋음  | 빠른 테스트                |
  | all-mpnet-base-v2                     | 768 | ⚡⚡ 중간  | ⭐⭐⭐⭐ 우수 | 추천 (Vertex AI와 동일 차원) |
  | paraphrase-multilingual-mpnet-base-v2 | 768 | ⚡⚡ 중간  | ⭐⭐⭐⭐ 우수 | 한국어 지원 좋음             |

In [10]:
# ========================================
# 1. 초기화
# ========================================
model = "all-MiniLM-L6-v2"
sentenceTransformer = SentenceTransformer(model)
print(f"✅ SentenceTransformer 모델 초기화 완료 ({model})")

qdrantClient = QdrantClient(":memory:")
print("✅ Qdrant 생성 완료 (메모리 모드)")

✅ SentenceTransformer 모델 초기화 완료 (all-MiniLM-L6-v2)
✅ Qdrant 생성 완료 (메모리 모드)


# 컬렉션 생성

In [11]:
# ========================================
# 2. 컬렉션 생성
# ========================================
collection_name="test"

if qdrantClient.collection_exists(collection_name=collection_name):
    qdrantClient.delete_collection(collection_name=collection_name)
    print("✅ Qdrant 기존 컬렉션 삭제 완료")

qdrantClient.create_collection(
    collection_name=collection_name,
    vectors_config={
        "dense": VectorParams(size=384, distance=Distance.COSINE)
    },
    sparse_vectors_config={
        "sparse": SparseVectorParams()
    }
)
print("✅ Qdrant 컬렉션 생성 완료")

✅ Qdrant 컬렉션 생성 완료


# 테스트 데이터 생성 (청킹 기반 워크플로우)

**워크플로우**: 긴 문서 → 청킹 → 임베딩 → 벡터 DB 저장

In [12]:
# ========================================
# 3-1. 긴 문서 샘플 정의
# ========================================

# 음식점 소개 긴 문서들 (여러 매장)
raw_documents = [
    """
    심야식당 기억 덕분에 남자친구하구 근사한 데이트 하구 가요!! 고등어봉초밥 진짜 맛나요 ⸝⸝ʚ̴̶̷̆ ̯ʚ̴̶̷̆⸝⸝ 고등어봉초밥 구운게 킥입니다
    숙성회도 도톰해서 넘 맛있어요 ,,,
    급하게 예약해서 방문했는데 프라이빗 룸으로
    안내해주시고 너무 친절하셔서 대만족!!
    주차는 좀 불편하지만 인테리어도 깔끔하고 조용하고 제가 좋아하는 째즈곡도 만족😄
    둘이서 조용히 먹기에 음식양도 적당하고 
    신선하고 맛있게 먹고 갑니다! 
    다음에 또 방문할께요!!
    부산에서 왔어요 두번째 방문이에요 
    좋은 기억이 있어서 다시 왔어요 
    너무 맛있습니다~
    짱짱!! 숙성사시미, 호소마키 너무 맛있어요. 
    도쿠리사케도 너무 맛있었답니다.
    직원분들 메뉴설명도 친절하고 분위기도 좋아요~ 분위기 좋고 맛있어서 데이트로 딱이에요 💓 너무 맛있어요…❤️ 분위기도 좋고 예약 하고 와서 기다리지 않고 이용 햇어요
    플레이리스트도 종아서 다음에 또 올게요🪽
    안키모마끼랑 부타가꾸니 강추…🤎 음식도 분위기도 좋아요! 전복이랑 사시미 맛있고 도미 마라도 맛있게 잘먹었어요 담에 데이트 할때도 올게요 고등어봉초밥은 비린맛없이 기름지고 너무 맛있고 안키모도 너무 고소하네요!! 오길 너무 잘했다는 생각이 들어요!! 하나하나 종류가 특별하고 너무 맛있어요👍🏻
    사시미도 신선하고, 살살 입에서 녹는 부드러움도 스릅🤤💦
    부위 하나하나 설명 해주시고 친절하세요💗 분위기 좋은건 기본이고,
    진짜 너무너무 맛있어용
    특히 고등어봉초밥!!!!!
    또먹고싶어요 ㅎㅎ
    다음엔 남자친구랑 가고 싶어용 음식도 맛있고 술잔도 고를 수 있는 기회가 너무 좋아요. 모든 안주가 고급지고 다 맛있고 다시 재방문할래요 고등어 봉초밥 예약하고 갔는데 모든 음식들을 다 맛있네요. 부타가 꾸니 모든듬사시미 서비스 주신 모든 음식이 다 맛있어요. 모두 추천합니다 고등어 봉초밥 최고 맛있어요..🙊 친구랑 맛있게 먹으려고 배고픈 거 참고 와서 여자 둘이서 메뉴 세개 시켰어요 ㅎㅎ 다 대만족이에요 !!! 또 오고 싶어요 💗 분위기도 좋고 얘기도 잘 들려요 >.< 사시미도 너무 맛있고 솥밥은 정말 정말 강추.
    너무 맛있었어요 ><
    회도 싱싱하구 퀄리티 좋고, 고등어 봉초밥 유명한 이유가 있었네요 너무 맛있어요!
    재방문의사 100프로입니다
    직원분들도 너무 친절하세요 처음가서 설렘반 걱정반이였는데 너무 맛있게 먹고 갑니다ㅠㅠ
    사시미는 말할것두 없고 고등어봉 초밥도 너무 조았어요🥺🥺
    왜 미리 예약 받는지 이해가 가는 맛이였어요!!
    잘 먹다 갑니당 음식이 너무 훌륭하고 분위기도 좋습니다. 재방문의사 200%에요! 고등어봉초밥 입문을 여기서 하네요. 안키모 김밥도 정말 훌륭해요!  고등어 봉초밥 비리지않고 너무 맛있어요 자극적이지않는맛 담에 또 올께요 분위기가 너무 좋고 음식이 다 맛있어요!
    특히 안키모 호소마끼 고등어 봉초밥은 꼭 주문하세요~!
    새유볶음면도 술 안주로 딱이에요!! 음식이 정말 맛있습니다. 고등어 봉초밥 예약 필수
    또올게요 맛있어요 안키모초밥과 숙성회. +매실하이볼 
    다음엔 고등어초밥먹으러 오겠습니다 저만의 또간집이에요🤤🩷
    너무 맛있어요 ㅎㅎ고향 친구들 데려와서 더 행복하네요
    좋은시간 보내게 해주셔서 감사합니다 :) 너무 맛있어서 한입먹고 바로 리뷰 쓰는중이에요🥹 진짜 미쳤어요..비린거에 엄청 민감해서 무서웠는데
    걱정한게 의미없을정도로 맛있어요!!!
    재방문할거에요 무조건!! 아직 메뉴가 다 안나왔지만ㅠㅠ 너무너무 맛있어서 기절할거같아요.. 두번째방문인데 더 자주 오게될것같습니다 신선한 재료에 깔끔한 손맛이 더해져, 가볍게 한 잔 하기에도 딱 좋은 스시 이자카야 매장 분위기도 좋고 음식도 너무 맛있어요! 고등어초밥 꼭 미리 예약하고 드세요!! 좋은 분위기에서 좋은 식사 하고 갑니다! 너무 잘 즐겼어요, 안키모호소마끼도요! 고등어 봉초밥이 정말 맛있습니다…..
    예전에도 왔는데 이맘때 되면 항상 생각나요 고등어 봉초밥 무조건 드세요!!!!
    분위기부터 서비스, 그리고 가장 중요한 맛까지 모든 게 완벽합니다 😊
    여자친구가 너무 좋아해요 😀 회사 사람들이랑 왔는데 너무너무 만족이에요
    제가 먹어본 고등어봉초밥 중에서 최고입니다!!! 꼭 또 먹으러 올게요ㅠㅠ 조용한 룸으로 와서 대화하기 좋았어요!! 그리고 고등어봉초밥이 진짜진짜 맛있어요!!!꼭 예약해서 먹으면 좋을것 같아요! 다음에 가족이랑 와도 좋을것 같아요! 친구 생일로 왔는데 룸도 너무 분위기 있고 좋고!!! 봉초밥도 짱짱짱 맛있어요  낙낙한 분위기와 정갈한 식사 너무 기억에 남을것 같아요 분위기 맛 전부 다 갖춘 곳 ㅎㅎ
    특히 고등어봉초밥은 필수로 꼭 드세요 !
    직원분들도 친절해서 기분이 더 좋았던 곳이에요
    음식 퀄리티 매우 훌륭하고 룸도 프라빗해서 좋습니다 ㅎ 예전부터 궁금했던 식당인데 드디어 방문했어용!!
    고등어 봉초밥이랑 사시미 옹김밥 모두 너무 맛있어요💓 목소리를 크게 내지않아도 되는 적당한 조용함이 얘기나누기 편해요 ㅎㅎ 숙성회가 참 맛있어요~~ 고등어봉초밥 맛집이라고해서 바로 찾아왔는데 진짜 맛있네요.. 올해 먹은 것 중에 제일 맛있어요.. 메뉴 구성들도 너무 좋아요 지금 먹으면서 실시간 리뷰 쓰는데 날 것/양념된 음식 조합이 좋아서 안 느끼하게 쭉 먹을 수 있을것 같아요. 고등어봉초밥 비린맛이 전혀 안나고 킹왕짱 맛있었습니다. 부타가꾸니의 루꼴라도 맛있었어요~ 분위기가 넘 좋아서 여자친구랑 다시 소개팅했습니다 기념일에 즐거운 식사였습니다
    고등어봉초밥, 안키모 호소마끼 너무 맛있어요  예전부터 오고싶었는데 드디어 🥺🖤,, 기대 이상으로 너무 맛있고 분위기도 좋아요! 데이트하러 와도 좋을거같아요 특히 솥밥 너무 취향저격,,🖤 데이트 코스로 추천합니다
    특히 고등어초밥은 꼭 드셔보세유~~~~
    한 입에!!!! 너무 맛있어서 행복합니다!😄
    다음에 또 방문하겠습니다 심야식당 말만 듣다가 오늘 와서 드디어 고등어봉초밥 먹었는데 너무 맛있고 맛있다는 말로 부족해요ㅠㅠ
    사시미도 엄청 신선하고 ,,,,, 아주 감동흐규 갑니다🩷 봉 초 밥은 예약해서 먹었어요! 음식 다 맛있고 직원분들 친절하세요! ^^ 재방문 의사 있어요  청첩장 모임으로 방문 했는데 분위기는 물론 너무 맛있어요🤗 방문 할때마다 칭찬받는 맛집 입니다! 
    연말 모임 장소로 강추해요~^^ 예약해서 왔는데 분위기도 좋고 음식도 너무 맛있아요오 처음 방문인데 맛있고 분위기도 좋아 재방문의사있습니다 인생 고등어봉초밥집이에요🐟
    여기 오려고 날씨 쌀쌀해질때까지 기다렸어요!ㅋㅋㅋㅋ 항상 다른 친구랑 오다가 오늘은 새로운 친구들을 데려왔는데 같이 온 친구들이 다 너무 맛있다고 해줘서 뿌듯했어요!
    항상 1층 바테이블이나, 2인석 앉았었는데 오늘은 룸으로 예약하고 안쪽으로 들어와서 앉았거든요 여기가 훨씬 프라이빗하고 조용해서 다음에도 룸으로 예약할 것 같아요ㅎㅎ 
    저 정말 1인1고등어봉초밥 하고 싶어요... 제발.. 테이블 한개 제한 없었으면 좋겠지만 그러면 세상 고등어 다 멸종됐겠지....🥺 기념일이라 연인과 첫 방문했는데 너무 좋은 경험이었어요!
    가게 분위기도 너무 좋고, 너무 조용하지도 시끄럽지도 않아서 편안하게 있을 수 있었어요!
    음식도 모두 맛있어서 배부르지만 않으면 더 먹고 싶을 정도ㅠㅠ
    다음에 또 방문하려구요!! 너무 맛나고 해방촌 데이트 후 저녁에 먹기 너무 좋은거 같아요!! 다음에 또 오겠습니다!  고등어봉초밥 하나도 안비리고 아주 맛있어요.
    그리고 에피타이저, 디저트도 주셔서 서비스도 매우 좋아요.
    해방촌 오시면 한번 오길 추천합니다. 사시미 고등어봉초밥 안키모호소마끼 먹었어요
    분위기듀 ,,,, 맛도 ,,,,, 비주얼도 정말 최고 🤍 너무너무 맛있어요! 사시미들도 정말 맛있고, 고등어초밥이 정말 맛있어요! 룸은 조용하고, 넓어서 편하게 먹고 좋았네요! 너무 맛있게 먹고 가요! 음식도 맛있고 분위기도 따뜻하구 좋아용 🐟 고등어 봉초밥은 여기가 제일 맛있는 것 같아요!! 꼭 예약하고 먹어보시길 추천드립니당 분위기도 적당히 조용하고 좋고 데이트 하기 좋은 것 같아요💕 너무맛있어요 ....! 사시미랑 고기랑 다 입에서 살살 녹아요 ㅠㅠ 봉초밥은 예약만 하면 되는건줄 알았는데 메모에 남겨야 하는거였더라구요 ㅠ 몰라서 못먹었는데 봉초밥 먹으러 또올거에요 분위기가 너무 좋고 음식도 정말 맛있어요 추천합니다 또오고싶네요ㅎㅎ 고등어봉초밥 너무 맛있어보여서 부산에서 일부러 이까지 왔어요
    역시 너무너무 맛있어요!
    사시미 먹으러 정말 많이 다녀봤지만 도미마라도 처음 먹어보는 조합이고
    배만 허락해 줬으면 전 메뉴 다 시켜봤을것...
    꼭 방문하세요!☺️ 고등어봉초밥, 사시미도 하나도 안 비리고 입에서 살살 녹아요 완전 맛있어요!! 찬 바람 부니까 오뎅도 필수.. 고등어봉초밥은 꼭 예약하고 오시는 걸 추천드려요ㅎㅎㅎ 인생 고등어봉초밥~~~~🤍🤍🤍 너무 맛있는 곳에서 기가막힌 고등어봉초밥...
    처음 먹어보는 어묵 스지탕 정말 맛있습니다 최고~! 봉초밥이 비리지도 않고 초밥 안에 맛이 잘 어우러져서 잘 먹었습니다ㅎㅎ 사진엔 없지만 어묵탕이랑 부타가꾸니도 맛있게 잘 먹었습니다 :) 고등어 봉초밥 진짜 기대했는데 맛있어요!
    사시미도 맛있는 부위만 주셔서 강추입니다👍👍 고등어봉초밥 먹으러 다시 한번 오려구요.. 한테이블에 한접시만 시킬 수 있는게 넘 아쉬워요😭 사시미도 신선하고 쫄깃해서 넘 맛있습니다!!!❤️ 매번 기념일 마다 오는데 올때마다 너무 좋아요
    좋은점 1. 음식이 너무 맛있음 신선한 재료로 최상의 맛을 끌어올림
    좋은점 2. 분위기가 기가막힘 친구 연인 모두 추천
    좋은점 3. 노래 선곡이 미쳤다 

    한번만 올 수 없어요  분위기 너무 좋고 음식도 맛있어요! 직원분들도 친절하셔서 다음에 또 방문하려고요 ㅎㅎ 분위기가 좋고 음식이 맛있어요 
    또 올래요~ 너무맛있습니다!!!!!!예약하고갔는데 룸으로 안내해주셔서 분위기도 좋았구요.    고등어초밥 진짜 맛있어요!!!!!!!!!!쵝오 작년에 한 번 와보고 반해서 벌써 각기 다른 친구들 데리고 4번은 왔는데 만족 안 한 사람이 없습니다!
    다른 음식들도 맛있지만 고등어봉초밥과 안키모호소마키는 진짜 무조건 드셔야합니다!! 요즘 봉초밥 맛있는 곳 찾기 힘든데 봉초밥은 무조건 여기로!!
    안키모호소마키는 처음먹어보는 맛인데 그냥 드셔야합니다!!
    그리고 이번에 망고맥주 처음먹어봤는데 음식이랑 정말 잘어울려요! 추천합니당! 친구들과 오게 됐는데 적당히 시끄러워서 대화하기 좋아요! 음식도 너무 맛있고 특이해서 기념일 혹은 친목으로 오시면 좋을 것 같습니다 ㅎㅎ 또 올게요! 하나도 안 비리고 너무 맛있어요 입에서 녹습니다 맛이 미쳤어요 고등어봉초밥 먹어도 먹어도 고등어가 계속 씹혀요 🐟🍣 고등어봉초밥 정말 맛있고 참깨가 들어간 게 다른 데랑 다르게 더 맛난 포인트인 거 같아여
    새우볶음면은 양도 많고 숙주도 아삭하고 배 채우기 딱이네요 저같은 맵찔이한테는 1단계도 매워요ㅎㅎ 음식 ,분위기 , 술 삼박자가 너무 잘 어우러지고 좋은 술과 좋은 안주 드시고 싶으신 분들이라면 강추합니다! 저번에 너무 맛있게 먹어서 예약하고 재방문 했습니당~ 안키모 호소마끼, 고등어봉초밥 모두 너무 맛있어요~ 안키모를 너무 좋아하는데 이렇게 듬뿍 먹을 수 있는 곳이 흔치 않아서 안키모 먹고 싶을 때 또간집으로 찾아옵니당. 고등어봉초밥은 생활의 달인에도 나오셨던 장인이시라 믿고 먹으셔도 됩니당 음식도 다 맛있고 직원분들이 설명도 친절하게 해주세요 핫플이자카야답게 분위기는 좋은데 요리의 퀄리티나 구성은 좀 아쉽고 사케서빙은 뭐 관련지식이 없는 수준이었습니다.  두 번째 방문했는데 역시나 기대 이상입니다! 사케 종류가 정말 다양하고, 요리들이 너무 맛있습니다ㅎㅎ 사장님 손맛이 좋은 것 같아요. 요리 하나하나 정성이 느껴지고, 플레이팅도 예뻐서 눈으로도 즐길 수 있었어요.

    특히 프라이빗룸 분위기가 너무 좋아서 편하게 이야기 나누기 딱 좋았어요. 조용하고 아늑해서 시간 보내는길 원하는 분들께 추천드려요!! 너무 자주오는 식당입니다...!!!
    정말 맛있구 고등어봉초밥은 진짜 서울에서 젤 맛있는거 같아요 .... 분위기도 좋고 룸은 조용해서 대화나누기도 너무 좋은 것 같습니다ㅎㅎㅎㅎ 저번에 방문하고 또 생각나서 왔는데 역시 맛있어요!
    또 방문드릴께요 미리 예약하고 온 보람이 있었어요..! 별도의 룸으로 안내 받고 조용한 곳에서 대화 나눌 수 있었어요 :) 기대했던 고등어봉초밥 강력추천! 아리아께 고등어초밥 보다 맛있습니다🤭 분위기부터 음식까지 전반적으로 만족스러웠습니다.
    모둠회도 숙성이 너무 잘되어있어 감칠맛이 느껴지고, 적당한 두께감이 있어 식감이 훌륭했습니다.
    당일 예약이라 아쉽게도 시그니처 메뉴인 고등어 봉초밥을 시키지 못했는데, 특히 안키모 김밥 너무 맛있었어요 환상적입니다.
    전체적으로 플레이팅, 맛, 분위기 모두 만족! 주변에 추천하고, 재방문의사 100%에요 맛있는 음식 만들어주셔서 감사합니다.❣️ 사시미가 정말 부드럽고 순삭입니당 메뉴 하나하나 맛있었고 특히 부타가쿠니가 인상적이에요! 나오는 채소들이랑 매쉬포테이토랑 같이 먹으면 조합이 너무 맛있습니당 분위기도 좋고 음식도 맛있어서 누구랑 와도 즐거운 시간 보낼수 있을것 같아요!! 망고 맥주도 외국 느낌 나고 시원 달달합니다 맛있어요
    겨울고등어라 크고 한입에 먹어야 맛있어요
    언제와도 기대를 저버리지 않는 최고의 봉초밥
    오는 날이 기다려져요~~ 연인이랑 함께와서 술 한잔하기 너무 좋은 분위기에요.
    안주도 맛있었어요!  봉초밥이 진짜 미쳤으니 꼭 예약해서 드세요 두번드세여 아니 세번! 음식은 빠짐 없이 맛있었으나 양이 적었고
    접시를 너무 빨리 치워서 무언의 압박감이 느껴졌어요
    생일 기념으로 느긋하게 먹고 마시고 싶었는데 
    금방 일어나 나왔습니다
    4번째 재방문중 역시 맛있고 데이트 겸 식사 및 술먹기 참 좋은곳 감사합니다. 음식의 맛은 물론이고 가게의 분위기, 메뉴에 없는 서비스 음식도 주시고 너무 잘 대접해 주셔서 행복한 시간을 보내고 왔습니다. 최고최고! 자주 오고 싶은 곳이에요 + 사케 잔을 고를 수 있는 것도 재미있었어요 :) 너무 잘먹고 갑니다 해방촌 맛집이네요!! 특히 고등어봉초밥 미쳤습니다🔥 N번째 방문중입니다 항상 너무 맛있고 친절하고 재방문의사 만프로❤️ 또 올게요!! 해방촌 가심 꼭 방문하세요. 인생 사시미랑 봉초밥 완벽합니다. 고등어 봉초밥 말이 안됩니다. 너무 맛있습니다 분위기 좋고 고등어봉초밥이 맛있어요
    다음에 또 방문할게요
    분위기 너무 좋고 음식이 너무 맛있어요! 비린내 없이 고등어의 고소함이 입안 가득 퍼져서 정말 감탄하게 되는 맛이에요. 분위기는 아담하고 조용해 데이트나 소규모 모임에 딱이며, 예약은 필수입니다. 고등어 봉초밥 외에도 다른 메뉴들도 너무 맛있어요!! 가게 분위기도 좋고 안주가 맛있어요 데이트 하기좋습니다 고등어봉초밥 비리지 않고 진짜 맛있어요 여자친구랑 분위기 내면서 한잔 하기 딱 좋습니다 ㅎㅎ 작년에 오고 넘 맛있어서 올해 또 왔어요 에피타이저 침샘 자극 1차 입안 홍수났구요
    숙성회 식감 혀 감각 다 살아났구요
    부타가꾸니 고기 촉촉 매쉬포테이토 은은한 치즈향 묵직하지만 부드러움 테토에겐 적절히 섞였구요
    마라도미는 겉바속촉 은은한 마라향에 고수랑 페스츄리 조합 훌륭합니다 여자친구 생일이라 예약하고 방문했는데 사시미가 정말 맛있고 좋은거 같아요 잘 먹고 갑니당 결혼기념일이어서 예약하고왔어요! 너무 맛있고 직원분들도 친절해서 넘 좋았어요! 담에 또 오겠습니다  늘 오고 싶던 기억이였는데 드디어 방문하였습니다! 모든 메뉴가 궁금해질 정도로 맛이 좋아요:) 분위기도 정말 좋습니당 가지튀김에 라구소스가 너무 잘 어울렸어요..!
    피자느낌도 나고 완전 색다른 음식이었습니다 분위기가 너무 좋고 음식도 넘 맛있어요❤️❤️❤️ 고등어봉초밥 예약해서 먹었는데 너무 맛있고 부타가꾸니도 입에서 녹아요!! 분위기도 좋아서 행복했습니다 ㅎㅎ 비린걸 정말 싫어하는데 시그니처인 고등어봉초밥은 끝맛도 비리지않았어요! 그리고 제가 살면서 먹어본 초밥 중에 단연코 최고로 맛있습니다. 비리지 않다니.... 정말 신기방기합니다. 여기 오시면 이 메뉴는 꼭 필수로 드셔보세요. 해산물 안좋아하는 남편도 극찬... 부타가꾸니도 고기가 정말 부드럽고, 함께 먹는 매쉬드포테이토가 킥이네요. 모임 있을 때나 외국 친구들을 여기 데려오면 좋아할 것 같아요~ ! 개인적으로 해방촌에 맛집 전멸이라 생각하는데, 이곳은 꽃이라고 볼 수 있습니다. 강추합니다!  고등어 봉초밥이 밸런스가 아주 짛습나다.
    꼭 예약해서 드세요.
    별관 룸도 아주 좋네요.
    재방문 의사 백프로입니다.
    회 맛있게 잘 먹었습니다!ㅎㅎ 재방문 의사 있어요 너무너무 맛있는 식사였습니다 봉초밥 예약무조건이고요 
    숙성회 봉초밥 하나도 비린거없고 좋았습니다 !!!!!! 
    가지라구도 !!! 생일이여서 왔는데 좋은 기억이 될거같네요  너무 맛있게 잘 먹었습니다! 분위기 맛 다 완벽했어요~~ 세 번째 방문이에요! 음식도 다 맛있고 술도 맛있고 짱입니다 회 선도가 좋습니다. 봉초밥 또한 비리지 않고 맛있습니다. 너무 맛있고 분위기 좋아요
    벌써 두번째에요 음식도 진짜 맛있고 직원분들도 다 친절하세요 
    다음에 또 방문하려구요 ㅎㅎ 고등어봉초밥 폭력적인 맛 입니다. 꼭 드세요 여러분 고등어봉초밥 달인이라더니 진짜 달인이세요
    대박 맛도리! 고등어초밥 최고!! 비린내 하나도 안 나고 너무 맛있었어요 :) 사시미랑 아귀간김밥도 흠잡을 데 없었습니다. 안키모 먹으러 왔는데 맛있어요!
    에타이저도 상큼하고 맛있었어요~ 분위기도 좋고, 모든 메뉴가 맛있었어요! 친구들과 오랜만에 행복한 시간 보내기에 즐거운 장소입니다 :) 고등어 봉초밥은 MUST입니다. 무조건 예약하고 드세툐. 너무 빨리 먹어서 사진도 없어요. 마라 도미도 아주 맛있습니다… 술안주로 최적! 무.조.건. 재방문합니다. 너무 맛있어요!! N번째 방문 중인데 또또 방문할거에요 히히 고봉초 짱!🤭 사시미 너무 맛있고 와인이랑 페어링이 잘 됩니다!
    안키모 김밥도 꼬소히고 신선하니 맛있어요 .. 먹자마자 음~ 맛있다가 절로 나옵니다 ㅎㅎ 
    마라도 맛있으니 드셔보셨으면 좋겠네요 ~  다 맛있어요! 맥주 회 모든게 신선했고, 스지오뎅탕 찍진 못했지만 맛있었습니다! 직원분들도 너무 친절하셨어요! 전복 아란치, 도미마라, 스지 어묵탕, 호소마끼...메뉴 하나하나가 예술적이예요. 주방에서 열심히 조리하는 직원분들 정성이 그대로 전달되었습니다. 분위기...감성 너무 좋아요. 또 올겁니다. 고등어 봉초밥 너무 맛있어요...
    사시미는 먹은 기억이 없어요 입에서 그냥 녹아버려서..
    그냥 드세요.!! 분위기가 좋고 메뉴가 특색있고 맛있습니다! 또 방문할게요! 주문한 모든 음식과 주류에 만족도가 너무 높아 특별한 날 다시 방문하려고 합니다
    질 좋은 사시미에 유자 사케 , 망고 맥주를 먹으니 젓가락이 쉴 틈이 없었습니다
    진짜 고등어봉초밥 미치도록 맛있어요
    회도 부드럽고 분위기 좋아요 
    날씨 좋은날 바 자리에서 창문 열때 한번 더 오고싶습니다 분위기도 좋고 무엇보다 음식이 너무 맛있어요
    무조건 재방문 의사 1000%입니다!!!!!
    특별한 날 와도 좋고 일상적인 날 와도 특별한 날이 될 것 같아요⭐️⭐️⭐️⭐️⭐️ 예약 풀이 었는데 운 좋게 빈자리가 나와서 예약하고 왔습니다
    조용한 개별룸 안내 받아서 넘 좋았어여
    신선하거 맛난 숙성회 등
    다 너무나무 맛있어요
    엄마랑 언니랑 대대 만족
    또 오고싶어여 직원분이 친절하시고 음식이 맛있어요. 제가 좋아하는 집입니다!너무 맛있어요 여자친구 생일 기념으로 방문했습니다.
    음식도 다 맛있고 서비스도 좋았습니다~!! 고등어봉초밥 먹어본것 중에 제일 맛있었구 안키모도 비리지 않고 맛있어요! 가지라구도 시켰는데 빵리필해서 꼭 드시길 ! 음식이 정말맛잇어요
    기념일 방문햇는데 점심으류 분위기도 맛도 귯귯

    술이랑 마시기도 너무즇을거같네오 고등어마끼 너무 너무너무 맛있어요! 그거 하나만 보고와도 되는 맛집. 꼭 예약하고 오세요 ◡̈ 사시미랑 고등어가 비리지 않고 부드러웠어요!
    식당의 분위기도ㅠ좋아서 도란도란 이야기할 수도 있고 사진도 잘 나와서 만족했어요~ 고등어 봉초밥 달인이라구 가게 문 앞에 있었는데 진짜 미쳐써요ㅠㅠ다른 곳은 밥이 많았는데 요기는 고등어에 밥이 있는 줄도 모르겠어요ㅠㅠ 평소 좋아하던 집이랑 사촌들이랑 가족모임으로 왔는데,
    다들 너무 맛있게먹어서 뿌듯하네요 ㅎㅎ
    사람 늘 많으니까 예약 필수!!❤️ 주변 지인한테 항상 추천하는 맛집이에요 ... 🤍 메뉴 하나하나의 풍미가 남다릅니다 🤤 다음에 또 올게요! 회가 다 신선하고 맛있고 분위기 좋고 직원분들도 친절하셔서 너무 좋아요☺️!! 벼르다가 왔습니다. 전날 봉초밥을 예약해야해서 더 특별했습니다. 모듬사시미엔 뱃다라쯔케가 따라오는데 이자카야에서 못봤던 광경입니다. 봉초밥은 조화가 정말 좋았고요. 안키모호소마끼는 쨍한 샤리와 안키모와의 호흡이 정말 좋습니다. 아란치니 역시 정말 말할게 없네요. 또올게요 플레이팅도 예쁘고 맛있어요:)
    하이볼 취향따라 달지않게 변경해주시는 것도 너무 좋네용😊 분위기 너무 좋아서 데이트 하기 좋아오!! 고등어봉초밥 너무 맛잇아요ㅎㅎㅎ 꼭 드세용 고등어봉초밥 진짜 최고입니다! 비리지 않고 고소하니 정말 맛있어요! 오뎅탕은 소쥬를 부르네여 ㅠ 분위기 맛 센스 뭐하나 나무랄 거 없는 취저 식당이에여~! 서버분들도 잘생기셨어요>.< 분위기도 좋고 카운터 앞 바테이블 앉았는데 음식 만드는 거 구경할 수 있어서 좋았어요
    회 + 안키모 호소마끼 조합으로 먹으니 너무 맛있어용 여자친구 생일에 두번째 재방문이에요 !
    올때마다 창가에 앉는데 분위기도 좋고 마싯어요
    고등어봉초밥 입에서 살살 녹아요 !!!!!! 숙성 회가 넘 훌륭합니다
    지인들이 너무 좋아해요 오랫만에 회사분들이랑 맛있는 저녁이네요!!! 회도 엄청 신선하고 고등어봉초밥너무 맛있어요!!!  꼭 시켜 드세요 음식이 담백하고 정직한 맛이에요. 자극적이지 않아서 늦은 시간에도 편하게 먹기 좋고, 재료 손질이 깔끔해서 믿고 먹을 수 있어요. 특별한 메뉴는 아니지만 한입 먹으면 괜히 또 생각나는 집이에요. 신선한 재료로 만든 초밥이 입안에서 사르르 녹아요🍣
    셰프님의 정성이 느껴지고, 하나하나 예술 작품 같아요.
    분위기도 아늑하고 직원분들도 친절해서 기분 좋은 식사 시간이었어요☺️
    다음에도 꼭 다시 방문하고 싶은 최고의 초밥집입니다! 🌸 고등어봉초밥은 제가 먹은 음식 중 제일 맛있어요. 오랜만에 올라온 가족들과 좋은 시간 보냈어요^.^ 봉초밥과 안키모 호소마끼 조합 짱이에요!
    아귀간이 듬뿍 들어가서 고등어 봉초밥에 얹어 먹어도 맛있네용! 도쿠리도 귀엽게 나와서 즐거운 식사였습니다 ;) 봉초밥은 꼭 드셨으면해요~
    정말 고소하고 비린맛없이 맛있었습니다!!
    고등어봉초밥장인👍🏻  음식이 모든것이 맛있고, 직원 분들이 친잘하심 기념일에 방문했는데 분위기도 너무 좋고 맛있습니다! 특히 고등어 봉 초밥이 진짜 인생 최곱니다!! 음식이 모두 깔끔하고 맛있어요. 직원 분들도 친절합니다 :) 음식도 넘 맛있고 분위기도 넘조아요.. 💖
    재방문 의사 백퍼입니다!!!!!!!  고등어봉초밥 너무 맛있고 내부도 분위기 있어서 좋아요 데이트하기에 최고에요 나만 알고싶은 식당이지만 이미 너무 유명한 심야식당🙂 고등어초밥🐟꼭 드세요 절대 후회 안하실거에요!! (1인1개 한정인게 아쉽) 모든 메뉴 너무 다 맛닜어요😍 데이트하러 왔는데 분위기도 정말 좋고 음식도 다 맛있어서 만족스러워요🫶🏻 고등어봉초밥 미쳤어요! 기름지고 맛있어요 ㅠㅠ 사시미는
    잘 모르겠고 고등어봉초밥 먹으러 또 와야겠어요  사시미 완전 부드럽고 입안에서 녹아요 ㅎㅎ 오이김밥도 아귀간이 진짜 맛있어요 강추!!  고등어 봉초밥, 안키모 김밥 제일 맛있어욥... 일본인 여자친구의 생일을 맞이해서 오게 되었는데, 양도 가격도 좋고, 무엇보다 맛있고 분위기 너무 좋아요🥹 가성비 매우 안좋음
    봉초밥하나 먹을만한 느낌 좋은 분위기 맛있는 메뉴 친절한 서비스
    재방문하지 아니 할 수 없네요 고등어 봉초밥은 제가 먹어본 것 중에 무조건 1등입니다..ㅠㅠ 다음에 또 온다면 무조건 고등어 봉초밥은 시킬거구 같이 시킨 마라볶음면도 너무 좋아서 여기 있는 메뉴 다 먹어보고 싶어요!! 사시미가 녹고 고등어 초밥은 말이 필요없어요 최고…💙💙 너무 다 친절하시고 분위기도 느좋이에요~~! 너무너무 음식이 맛있어요 
    전화해서 고등어봉초밥 꼭 예약하세뇨..
    마라새우볶음면 첨 먹겄는데 너무 맛있고, 이번엔 솥밥 기본으로 먹었는데, 이것도 맛있었어요..!! 진짜 레전드 무친놈 식당.. 저 원래 리뷰 안쓰는데 너무 맛잇어서 남깁니다...😭😭😭😭고등어봉초밥 솔직히 다른데랑 비슷할 줄 알았는데 절대 ㄴㄴㄴ 한입 먹으면 심봉사 바로 시력 2.0되는 맛이고요 전메뉴 다 맛있어요.... 해방촌 올 일 있을때마다 오고 싶을 것 같아요 🩷🩷🩷🩷 서울 고등어봉초밥 맛집인 곳이라 너무너무 먹어보고싶었는데 정말 맛있어요....🫠 부타가꾸니도 너무 부드럽고 살짝 짭조름한데 메쉬포테이토랑 루꼴라랑 너무 잘 어울려요 💗!!  먹으러 또 올 의향이 진짜 있습니다 .... 😇 음식이 진짜 맛있고 산토리맥주도 깔끔히니 맛있습니다. 다음에도 꼭 재방문할게용 분위기 좋아요 음식 맛있어요 숙성회 첨먹어보는데 맛나요 언제나 좋은 서비스와 맛을 보장하는 곳
    칠하게 2차로도 좋고 1차로와서 다 맛봐도 거를 타선이 없다 지난번 워크인으로 방문하고 너무 마음에 들어서 이번엔 예약하고 방문했어요~ 여전히 사시미는 너무 맛있고, 부타가꾸니는 이번에 처음 먹어보는데 입에서 녹아버려요오~~! 다들 해방촌 오시면 꼭 들려보세요~!! 분위기 좋고 맛도 좋고 직원들도 친절해요 스지 맛있고 봉초밥이 진짜 맛있어요 고등어의 향은 가지고 있지만 비린내는 나지 않습니다. 1인1메뉴 필수사항이라 양 적으신 분들은 배뷰를 수 있지만 진짜 맛있습니다! 예전부터 꼭 방문해보고 싶던 곳인데 요번에 와보았어요,, 기대한것보다 훨씬 맛있습니다,, 여긴 무조건 고등어 봉초밥 진짜 기본으로 시켜야됩니다 비린맛 1도 안나고 큼지막하게 입 안을 가득 채우는 풍미가 대박이에요; 걍 모든 메뉴가 완벽.. 분위기도 좋고 담에 꼭 또오고 싶은곳이에요!!! 고등어봉초밥 예약해야 먹을 수 있는 곳이라 기대하고 예약했는데 비린맛 없고 맛있어요👍 고등어봉초밥 너무 맛있구
    부타가꾸니도 고기가  입 안에서 살살 녹아서
    부드럽게 잘 넘어가요!!!!
    예약을 미리하고 왔는데 별도로 마련된 별관에서 
    조용하게 즐길 수 있어서 좋았습니다
    맛있는 이자카야를 찾다가 우연히 발견해서 방문하게 되었는데 원래 인기가 많았던 매장이었어요! 고등어봉초밥이 맛있다고 해서 예약해서 먹어봤는데 기대 이상이었어요,,원래는 잘 못먹는데 여기는 하나도 안비리고 맛있기만 했어요! 다들 한번씩 드셔보세용..인기가 많으니 다들 예약하고 오시는 걸 추천드립니다ㅠ 분위기가 좋고 사시미가 너무 맛있어요 ㅜㅜ 다음에 해방촌 놀러오면 다시 방문할게요! 여자친구랑 기념일로 왔는데 분위기도 너무 좋고 음식이 정말 맛있어요. 음식 설명도 친절히 잘 해주셔서 만족스러운 시간을 보냈습니다. 특히 사시미와 고등어봉초밥을 한입에 먹고 기억사케를 한잔 하면 캬..  매번 올 때마다 정말 잘 챙겨주셔서 감사해요! 또 방문하게 싶게 만드는 맛과 서비스! 고등어봉초밥은 사랑입니다🧡  살면서 먹어본 고등어 봉초밥 중에 제일 맛있었어요. 
    마라 볶음면도 특색있고 굿
    전반적으로 음식 맛이 좋습니다
    공간은 큰 창을 열어놔서 환기도 잘 되고 괜찮아요 캐치테이블에 리뷰가 좋아서 예약을 했고 방문을 했는데 역시나 너무나 맛있었어요 !
    룸이 조용하고 서비스도 좋고 너무 맛있어서 주변에 올 일이 생기면 다시 방문할 것 같아요 추천이요 진짜 다 너무너무 맛있어요!!!
    메뉴 하나하나 다 강추합니다.
    이번에 봉초밥 예약을 못해서 못먹었는데 다음에는 꼭 예약해서 먹어보겠습니다 룸이 진짜 좋은 것 같아요! 음식 하나하나 너무 맛있고 고등어봉초밥이 어디서 먹었던 것보다 제일 맛있습니다ㅠㅠ 데이트하기에 최고!! 미리 예약하세용 주차가 어려우니 대중교통 이용을 추천합니당🙈 고등어회는 죽어도 못먹는데 ㅋㅋ 오늘 처음 다 먹어봤어요. 고등어 초밥 먹으러 또 올게요. 2번 주문 못하는게 너무 아쉬웠어요. 🤣🤣 고등어 봉초밥 사시미의 퀄리티와 맛이 정말 맛있었습니다
    다음에도 또 방문할게요!ㅎㅎ 분위기좋고 맛있어요!
    룸은 또 분위기가 좋아요 :) 분위기도 조용하고 너무 좋고. 특히 음식이 하나도 빠짐없이 다 맛있어요...🥹 재방문 꼭 할게요❤️‍🔥❤️‍🔥❤️‍🔥 낢시가 좋아서 창가석에 앉은게 빛을 발했어요!
    고등어봉초밥 이거 테이블마다 있던데
    왜 사람들이 다 주문하는지 알것 같아요!
    생선류나 초밥 싫어하는 분들도 드시면 좋을 것 같아요
    꼭 주문하길 추천!!!
    마라 새우 볶음면도 맛있게 먹었습니다:) 다ㅜ맛있는데
    고등어 봉초밥이 진짜 넘사로 맛있습니다!!!!!
    한접시 더 먹고싶은 맛 ㅜㅜ 고등어봉초밥, 안키모 모두 재료가 신선한 맛이 잘 느껴집니다.
    재방문의사 무조건 있습니다 !! 너무 맛있고 분위기도 좋았어요 다음에 또 오고 싶어요! 룸이 완전 프라이빗해서 좋았어요 🍀🍀 봉초밥 진짜 괜찮네요 ㅎㅎ 너무 오고 싶었던 곳이어서 전화로 예약해서 왔는데 고등어 봉초밥은 꼭 예약해서 드세요!!
    다음에 오면 고등어 봉초밥 2개 예약할거에요..
    또 오고 싶은 곳입니다🥰 남자 친구 생일이라 특별한 곳을 찾다가 오게 되었어요🥰
    기대했던 만큼 정말 맛있네요!!
    혹시 메뉴 고민되신다면 이대로 시켜보세요!! 정말 메뉴 하나하나 전부 만족했습니다
    계절 사시미 1인
    고등어 봉초밥
    안키모 호소마끼
    새우 마라 볶음면

    남자 친구는 특히 고등어 봉초밥이 너무너무 맛있다네요 ㅠㅠㅎㅎ
    덕분에 즐거운 데이트 하고 갑니다🥰 분위기 좋고 음식이 너무 맛있어요 😸🤍👍
    고등어 최고에요~~🐟 가게가 일단 너무 깔끔하고 쾌적해요! 바테이블에 앉는거 추천합니다 ㅎㅎ 음식더 너무 맛있고 직원분들 모두 친절하세요 두번째방문인데 앞으로도 계속 오고싶어요 예전에 지나다닐 때마다 한 번쯤 방문하고 싶은 식당이었는데, 운 좋게 가족 식사를 할 수 있는 기회가 생겨 식사할 수 있었습니다!

    회도 맛있고, 특히 ⭐️고등어 봉초밥⭐️이 정말 맛있네요~!! 예약하실 때 반드시 주문 넣어주시길 강추합니다!! 마라 볶음면도 정말 맛있습니다. 너무 맛있게 잘 먹었습니다! 분위기도 좋고 고등어초밥이 진짜 맛있어요!! 노무 완벽해요…. 진짜 오마카세보다 더 맛있고 고등어봉초밥은 미친줄알았어요 제발와주세요 또올게여 사랑합니다 요리해주셔서 감사해요 모든 메뉴 하나하나가 다 맛있고 서비스도 친절해서 좋았어요ㅎㅎ 다음에 또 와서 푸파할 예정입니다 :) 분위기 느좋 맛도 너무 맛있어서 메뉴 다 먹어보고싶어요!! 데이트장소로 추천드립니다🩵 두 번째 방문인데 조용하고 맛있어요 🤭 여태먹은 고등어봉초밥 중에 제일 맛있습니다🐟🍣 어묵탕도 너무 알차게 나오고 하이볼도 맛있어서 재방문의사 100%입니다. 다음에 또 올게요! 언덕 올라 별실(?)인데 너무 좋았어요 :^) 고등어봉초밥이랑 오뎅탕 너무 맛있어요! 남자친구랑 여행겸 방문했는데 너무너무너무 맛있고 분위기도 최고로 좋고, 너무 조용하거나 시끄러운게 아닌 일행과 대화도 가볍게 나누며 행복하게 먹을 수 있는 공간입니다! 고등어봉초밥은 말해뭐해.. 숙성사시미 진짜 맛있어요 진짜로 그냥 녹아요 꼭 드세요.. 인테리어도 좋고 음식도 맛있어서 또 오고 싶어요~ 특히 고등어봉초밥 대박! 맛있습니다!! 회도 맛있고 도미마라도 맛있네요ㅎㅎ 고등어 꼭 드세요! 분위기 좋고 음식도 맛있어요ㅜㅜ 매년 1회 꼭 방문할게요! 벌써 4번째 방문인데 너무 맛있어서 앞으로도 계속 갈 거 같아요 공간도 너무 예쁘고 분위기도 좋아서 데이트 장소로 추천해요~,~ 음식들도 흔하지 않은 메뉴인데 맛있습니다ღ 너무 맛있고 음식이 독특하고 좋았습니다…
    특히 가지라구에 같이 나오는 바게트는 진짜 역대급 1등 바게트 수준… 너무 맛있습니다 강추 드려요!! 너무 맛있어요 담에는 고등어봉초밥이랑 안키모 꼭꼭 먹으러 올여구요!!! 유자사케 꼭…드세요 처음부터 큰용량으로 시키세요…저희처럼 작은거 두번시키지 말고요 꼭……⭐️ 분위기도 좋고 가족들끼리 왔는데 너무 마음에 들어 했습니다. 특히 고등어봉스시로 유명한데 정말 맛있더라구요. 평소에 고등어를 잘 안먹는데 먹는게 아까울 정도로 맛있었습니다! 잘 먹고 갑니다. 제가 먹어본 시메사바중에 제일 맛있어요 ㅜㅜ 시메사바 안좋아하는데 진짜 최고임니당... 다른거도 맛있어여 너무 맛있네요~~ ㅎㅎ비린내도 하나도없이 순삭했어요^^ 강추!!! 입에서 멜팅하는 느낌이예요~  예약해서 왔는데 왜 인기 많은지 알 것 같아요. 고등어 봉초밥 비린맛 없이 깔끔했고, 특히 안키모 호소마끼가  맛있었어요 추천합니다b 고등어봉초밥이 맛있고 가게 인테리어가 예쁘네요
    식기도 특이하고 예뻐서 기분이 좋아지네요 잘 먹고 갑니다 사시미와 안키모 너무 맛있었습니다! 창가 자리에서 선선한 가을 날씨에 완벽한 식사였고, 친절한 서비스까지 흠잡을더 없었습니다. 다음에 또 방문할게요~~ 오늘 또 취하고 갑니다
    인테리어에 취하고
    안주에 취하고
    사케에 취하고
    마지막 디저트까지 친절하게 챙겨주심에
    취해 맛있게 잠듭니다
    감사해요! 사시미 회가 크게 썰려있어서 더욱 씹는감이 있고 좋았어요 숙성회같이 쫄깃하고 부드러워서 먹을때마다 놀랐습니다! 특히 예약하고 먹었던 고등어봉초밥은 정말..맛있었어요🤍 참깨를 안 좋아하는데 이렇게 먹으니 거부감이 없어 고소하니 너무 좋더라고요 새로운 제 식견을 넓혀주셨어요✨❕ N번째 방문입니다
    소중한 친구들만 자신있게 데려오는데 올때마다 먹고 만족해주면 제가 좋습니다
    고등어봉초밥 서울에서 맛있다는 집 많이 먹어봤는데 제일 맛있습니다
    이외 메뉴도 다 맛있어요 오래 장사해주셨으면 좋겠습니다 봉초밥이 맛있다는 소문으로 연차쓰고 왔습니다! 
    좋은 자리와 서비스로 즐거운 시간 보내다 갑니다!
    물론 음식도 정말 맛있습니다!
    분위기도 너무 좋고 직원분들도 친절하십니다
    모듬 사시미 살살 녹고 고등어 봉초밥은 장인정신이니 꼭 드셔보세요! 추워지는 날씨에 사케한잔 하기 딱인 곳입니당 모든음식이 다 너무맛있었습니다! 추천해주신 사케도 굳이었어요 간만에 찾은 심야식당 기억
    언제나 느끼던 맛 그대로라 만족스러웠어요.

    항상 이 퀄리티가 유지되면 좋겠어요 사케 잔 고를 수 있는게 너무 귀여워요!!
    고등어 봉초밥 진짜 최고십니다 .. 😭
    다른메뉴도 다 맛있었고 룸 분위기 조용하고 프라이빗해서 좋았어요  너무 맛있고 분위기도 좋고 친절해요👍🏼 처음방문해본 심야식당 직원분들도 친절하게 맞이해주시고 고등어 봉초밥은 듣던대로 맛이 너무 좋았어요 좋은 기억 남기게 해주셔서 감사합니다 고등어봉초밥 예약해서 왔는데 넘 맛있어요~ 예약해서 룸에서 아늑하게 드세요~ 서울 여행 올때마다 방문하는 곳 입니다 ❤️ 
    벌써 3번째 방문 ㅜㅜ 
    고등어봉초밥 먹어본 사람이라면 주기적으로 와야하는곳이에요 !! 
    다른 메뉴도 너무… 다 전부 맛있고 
    조용하고 친절하시고 너무 행복한 공간 입니다 
    💗💗 또또또간집 💗💗  넘 맛있고 분위기 좋아요~ 바 자리도 편안합니다. 데이트하기 분위기 좋았어요. 새우 마라면은 1단계도 좀 많이 얼얼한 편이라 주의하세요~^^ 너무 맛있게 잘 먹었습니다 직원분들도 너무 친절하십니다 !! 달인님이 확실합니다! 고등어초밥과 사케 넘 맛있게 먹어서 일주일 지나기 전에 또 왔어요! 정말 맛있고,,분위기도 좋고 넘 좋네요 무슨 이름이 긴 도쿠리 사케 정말 맛있으니 대추천드립니다~~~ 친구가 맛있다고 또 가자고 해서 왔는데 고등어봉초밥도 너무 맛있고 사케도 종류가 많아서 넘 좋네요...  특별한 날에 꼭 방문하는 해방촌 최애맛집 심야식당 기억❤️
    고등어봉초밥 꼭 주문하세요. 어딜 가도 여기가 제일 맛있습니다!!!
    다른 요리도 먹어봤는데 단 하나도 빠짐없이 맛있어요.
    가게 분위기도 좋고 직원분들도 다 너무 친절👍 고등어봉초밥 최고 🥹 해방촌까지 들어온 가치가 있네효 사시미도 맛있고 고등어봉초밥 최고로 맛있어요 꼭 예약하고 드세요 진짜 맛있습니다 어머니 생신 기념으로 방문했는데 음식도 너무 맛있고 분위기도 너무 좋아요. 또 방문할게요 !! 고등어 봉초밥 꼭 예약해서 드세요 청첩장모임하기 너무좋아요 맛있습니다! 낮에 방문했는데 맛도 좋고 적당히 조용하고 밖도 구경하기 좋아요~ 고등어 봉초밥이 안비리다는 후기를 봤었는데 진짜 비리지 않고 고소하고 기름져서 맛있어요! 매장도 예쁘고 요리도 너무 맛있어요!! 사시미도 아주 맛있습니다!!! 창가에서 시원하게 식사할 수 있어서 너무 좋았고 고등어초밥, 부타가꾸니 너무 맛있게 잘 먹었어요! 강추!!! 캐치테이블을 통해서 심야식당 기억을 알게되었고 기념일 맞이겸 방문했는데 너무 좋았네요🥰 사시미도 신선했고 전복요리도 너무 맛있어서 남편이랑 잘먹었습니다! 룸도 개별룸으로 안내해주셔서 너무 좋았네요 그리고 서비스 주신 음식들도 너무 맛있었습니다 ㅎㅎ 추천드려요 ㅎㅎ 숙성회 먹고 소리지름;; 개맛돌희 회전문가가 숙성시킨게 분명;;; 거기에 분위기도 데이트하기 딱 좋슴다 재방문 10000퍼 남자친구 생일 데이트로 방문해서 봉초밥이랑 가지라구 시켜서 먹었어요! 봉초밥 완전 좋아하는 봉초밥인데 봉초밥 너무 맛있어서 깜짝 놀랐어요! 기념일 생일 데이트로 강추 드립니다 ! 안키모 최고에요.
    봉초밥도 예약 해서 먹으시길 추천해요🙂 분위기가 너무 좋고 술 종류도 엄청 다양해요!!!
    사시미가 진짜 미친맛이네요…… 🤤
    직원분들도 친절하시구 꼭 다시 와보고싶은 곳이예요 ❤️ 너무 맛있어요!!  다시 방문하겠습니다~ 고등어봉초밥 원픽!! 안키모랑 같이 시켜서 먹으면 말다했,,!

    코스도 완벽했고 직원들도 세심하게 안내해주셨습니다

    다음에도 또 올게요! 중요한 미팅 있을때마다 자주 오는 곳입니다!
    분위기도 좋고 음식도 정말 맛있어요 고등어 봉초밥 미리 예약하고 왔어요🩷 친구 생일 맞아 오랜만에 방문했는데 항상 한결같이 맛있네요! 오늘도 잘 먹고 갑니다💗💗 아 이것이 요리구나 싶은 생각이 드는 가게입니다😀
    식감과 맛이 각 재료의 특성에 맞게 레이어드 되는 요리입니다. 이곳에서만 맛볼수 있기에 다시한번 방문하고 싶은 곳입니다.
    사케도 추천 부탁드렸는데 저희가 원하는 사케로 정말 딱 맞게 친절하게 주셨습니다 최애식당 기억... 고등어봉초밥이랑 안키모 강추입니다!!! 생일을 맞이해서 평소에 좋아하는 가게를 방문했어요! 여전히 숙성회는 맛있고 봉초밥도 최고입니다!👍 와이프 생일에 방문했는데 너무 분위기도 좋은데 날이 쌀쌀한 편이였는데 히터도 살짝 틀어주는 배려에 너무 감사했습니다🙆🏼‍♂️😁
    고등어 봉초밥도 너무 맛있었는데 새끼참치회를 처음 먹어서 그런지 너무 맛있게 먹었습니다!
    기념일을 더 풍성하게 만들어주셔서 감사합니다👍 4번째 방문인데 한결같이 음식 넘 맛있고 분위기도 좋슴니당 세번째 방문!! 꼭 고등어봉초밥 예약해서 오세요 !!
    먹어본 고봉 중에 젤 맛있어요ㅠㅠ 안키모호소마키도 녹진하고 너무 맛있어요! 담에 또 올게요🐟 고등어봉초밥이 먹어본거 중에 제일 맛있어용 ㅜㅜ 하나도 안비리고 완전 부드럽고 맛있어요!!!! 꼭 예약해서 드세요!! 생맥주도 맛있고 분위기도 좋아요❤️ 음식의 퀄리티도 너무 좋고 분위기도 데이트하기 너무 좋은 곳이라 아주 만족하고 주기적으로 방문 할 것 같아요 정말 너무 맛있어요ㅎㅎ 사시미도 엄청 맛있고 개인적으로는 안키모 호소마끼가 정말 맛있어요ㅎㅎ 정말 잘 먹었습니다! 비오는날 창가에 앉았는데 음식도 맛있고 분위기가 너무 좋아요~! 벌써 4번째 방문이에요! 고등어봉초밥을 항상 시키는데 항상 맛있어서 너무 감동적입니당..💖 도미마라는 처음 주문해봤는데 고급스럽고 맛있네요!! 제가 좋아하는 망고맥주가 있어서 너무 반가웠어요! 데이트 맛집으로 추천드려요ㅎㅎ 분위기도 좋고 와인이 너무 맛있어요~
    음식도 다 맛있어요!
    친구랑도 데이트로도 좋은 곳 같아요~ 예약하고 방문했습니다.  아담하지만 분위기 있고 해산물 베이스의 음식이 맛있었습니다.  창가자리에 앉았는데 분위기 너무 좋고
    음식도 하나같이 다 맛있어요!
    데이트 장소로 완전 추천합니다!!!!
    다음에 또 방문할게요 🫶 분위기 맛 뭐하나 빠진는곳 없는 훌륭함 맛집!
    분위기도 좋고 고등어봉초밥 너무 맛있어요..❤️ 처음 캐치테이블통해 예약하고 갔는데 분위기며 맛이며 너무너무 맛있었어요~ 먹는순간마다 눈이 번쩍하게 만드는 맛
    생활의 달인에 나온집이더라고요...
    근데 예약하고도 몰랐네요😅

    2주년 기념으로 남친몰래 예약하고 갔는데
    너무너무 성공적이고 잊지못할 기념일이 되였네요👍👍👍🫶 너무 맛있어요...!! 뷴위기도 좋아요 :) 재방문 했는데 여전히 고등어 봉초밥은 최고입니다!
    부타가꾸니의 퓨레는 아주 부드럽고 처음 먹어보는 맛이며 돼지고기와 아주 조화로워요.
    다음에도 재방문 하려고요👍🏻👍🏻 맛 가격 서비스 다 좋습니다.
    와인도 소주 사케 모두 훌륭하고 
    음식너무 맛있습니다.♡ 이렇게 분위기 좋은 룸이 있는줄 몰랏네요 :)
    빠른 안내와 친절한 서비스 분위기도 좋아요!
    최애는 안키모, 숟가락으로 퍼먹어서 너무 좋아요.. 친구따라 왔는데 너무 좋아요
    분위기도 넘 좋고
    일단 안주가 최고여요 

    고등어초밥 친구가 먹으라했는데,,, 이거 진짜 예약할때 꼬오오오옥 시키세요 🐟 
    저는 담에 남자친구 델꼬 올게요~~~~~~~!!  첫 방문인데 분위기도 좋고 안주도 너무 너무 맛있네요!
    또 방문하고싶습니다.:) 숙성회 맛있습니다, 기본적으로 시킨 음식들 다 맛있어요
    서빙해주시는 분들 음식 앞에서 큰 소리 내시지 않고 조근조근 설명해주시고, 언더락으로 마실 때 얼음 리필도 말씀안드려도 알아서 해주시더라구요.. 세심한 부분이 만족도를 높이는 포인트인 것 같습니다
    다음엔 고등어 봉초밥 예약해서 먹으러 오겠습니다 마라면과 솥밥 모두 간도 적당하고 깔끔하게 맛있었어요!
    하이볼도 모두 너무 맛있었습니다 :)
    기억솥밥 제 최애♡ 여자 2명이서 왔는데 배부르게 먹구 가용. 와인도 리슬링으로 먹었는데 적당히 달달하고 회랑 넘 잘 어울려서 와인 리스트도 좋은 거 같아요(잘 모루지만)😆 재방문의사 100%입니다!!! 고등어봉초밥이 먹어본 고봉중에 젤 맛있어요!! 디저트까지 나오고 분위기도 좋고!! 데이트로 왕추천해요 음식이 다 너무 맛있고
    고등어 봉초밥은 꼭 드세요! 분위기, 서비스 모두 너무 만족했어요! 사시미도 비리지 않고 맛있어요👍🏻
    소금 찍어먹는거 너무 좋아하는데 확실히 맛집인게 소금 옵션이 있어요 ㅠㅠ 최고❤️ 또 올게요!! 음식 다 너무 맛있고 직원분들 친절하셔서 기분 좋게 즐기고 가요! 음식도 너무 맛있고 분위기도 너무 좋아요!!❤️❤️❤️❤️ 너무 맛있어요 맨날 웨이팅해서 못먹다가 드디어 먹어요 감사합니다ㅔ 고등어 초밥 사시미 다 너무 맛있아옹 
    가지 라구 먹으러 또 올게영!!! 스시 2인 세트와 가지 라구 시켰는데 조합이 너무 좋았어요. 하이볼과의 궁합은 말할 것도 없구요. 룸 이용했는데 조용하게 이야기 하기 좋았어요. 재방문 의사 100% 도로변에 있는데도 차소리 없이 조용해서 대화하기 편해요
    조명 밝기도 적당해서 아늑하게 식사하기 좋을 것 같아요

    음식도 맛있고 특히 고등어봉초밥 추천합니다 꼭예약하세요 고등어봉초밥이 부드럽고 풍미가 좋아요. 맛있고 분위기도 좋아서 특별한 날에 친구들이랑 오기 너무 좋아요🥰 아니 고등어 봉초밥 진짜 너무 마싯소요 입안에서 기름+육즙 녹아요 먹고있는데 또 먹고싶따 음식들 하나하나 퀄리티가 엄청나요! 맛은 말이 필요없네요👍🏻 매장 분위기도 너무 좋고, 와인 등 술 종류도 다양하고 특히 논알콜 음료도 있어서 좋았습니다. 분위기있는 곳에서 좋은 시간을 보낼 수 있어서 좋았습니다  🥰 고등어 봉초밥 최고에요!!👍🏻👍🏻👍🏻 이제까지 먹어본 것중에 제일 맛있습니다. 또 오고싶어요 음식 퀄리티가 너무 높고 분위기도 좋아요 ~ 재방문 의향 있습니다 ! 진짜 제 인생 맛집이라 아무한테도 안알려주고 싶지만, 좋아하는 사람들이랑은 무조건 꼭 한번은 오게되는 곳이에요!! 

    특히 고등어봉초밥이랑 안키모호소마끼 진짜 세상에서 젤 맛있어요👍🏻👍🏻 인생맛집입니다 오이김밥 없어지는게 아까워서 못먹고있는중 ㅠㅠ  올땨마다 느끼는 거지만 고등어 봉초밥과 사케가 너무 좋습니다. 한정판 사케가 많고 추천도 너무 잘해주셔서 항상 최고의 경험을 하고 가는것 같습니다 음식이 너무너무 맛있고 분위기도 좋아요~ 2번째 방문하는데 분위기가 너무 좋아요! 메뉴들도 다 맛있습니다 ㅠㅠ😊👍 너무 맛있고 실내 분위기도 좋습니다!  분위기도 너무 좋고, 특히 특별한 날에는 룸으로 예약하시면 훨씬 더 좋을 것 같아요! 음식은 고등어봉초밥 꼭 전 날에 예약하시구 드셔주세요ㅠㅠ 너무 맛있어요!! 다음에 또 방문할게용!!~ 고등어봉초밥 진찌 마싯움…👍🏻 비린맛 하나도 안 나구 꼭 드세여 너무 맛있어요!!!!! 고등어초밥 유명한 이유가 있네요 ㅋㅋ 제주도 보다 맛있는듯 
    그리고 술 좀 먹고싶은 날이다? 도미마라 무조건 ㄱㄱㄱㄱ 직원분들도 친절하시고 음식도 맛나요 ㅎㅂㅎ 생일 기념으로 왔는데 맛집찾아서 기분 좋아요 다음에 또 올게요!
    고등어봉초밥 먹고싶을때마다 항상 오는 곳입니다.
    안키모도 너무너무 맛있어요!!!!
    항상 감사합니다~! 맛있어요ㅠㅜㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ
    예약해서 편하게 즐길수 있었고 분위기도 좋아서 다음에 또 올 예정입니다.  만족해요 :) 고등어 봉초밥, 회, 마라도미 모두 다 맛있었어요!!! 선선하게 창가에서 친구랑 분위기 좋게 즐길 수 있어서 추천합니다!! 고등어 봉초밥은 진짜 맛있어요
    나머지 음식들도 말해 뭐해요 ㅠㅠㅠㅠㅠㅠ
    좋은시간 보내고 왔습니다 또 갈거예요🤍 고등어봉초밥이 먹고싶어서 재방문 했어요!! 역시나 훌륭하네요. 음식하나하나 퀄리티가 너무 좋아서 또 방문할 예정입니다 ㅎㅎ 추천! 시그니처 메뉴인 고등어 초밥 너무 맛있게먹었습니다~~
    분위기도 좋고 
    음싣 양도 적당합니다 사시미 너무 신선하고
    에피타이져가 진짜 맛있어요
    서비스 좋고 분위기도 좋아요 조용하고 음식도 맛있고 특색있어서 친구랑 좋은시간 보냈습니다 !!!!! 추천추천 여자친구 생일이라서 방문했는데 너무 좋아했어요. 사시미는 숙성이 제대로 돼서 찰져요. 솥밥은 밥과 생선의 고소함과 파의 단맛이 잘 어우러져서 끝내줍니다. 매장 분위기도 좋고 대화하면서 술 마시기 좋아요! 친구들이랑 왔는데 음식이며 분위기며 다 좋았습니다! 재방문 의사 100% 음식도 맛있구 분위기도 너무 좋아요!! 특히 고등어봉초밥 진짜 너무 맛나요 ㅎㅎㅎ 고등어봉초밥 예약하고 먹었는데 입에서 사르르르 녹아버리네용 ! 사케랑 같이 먹었는데 맛있어요 회가 입에서 살살 녹아요... 분위기도 좋고 맛있습니다!

    꼭 사시미 드시길 바랍니다... 입에서 녹아요....

    술잔도 예쁘고 직원분들도 친절하세요 ㅎ_ㅎ 친구가 와보고 맛있다그래서 예약하고왔는데 너무맛있네요~~ㅎㅎ 생활의달인 고등어봉초밥 보고 벼르고 있다가 이제야 방문했는데 기대를 져버리지 않는 맛이에요!! 고소한 고등어 향이 가득 퍼지는게 술을 부르는 맛. 고소한 안키모 아귀간 좋아하시면 오이김밥도 무조건 추천드립니다. 비오는 날에도 창가 풍경 보며 먹기에 분위기 굳. 추우시면 담요 가능하니 참고! So far we've had four things off the menu and everything has been wonderful. Nice atmosphere. I'll definitely be back again soon.  삼시세끼 365일 먹어도 질리지 않는곳  고등어봉초밥이랑 안키모 너무 맛있고, 분위기도 좋아서
    다음에도 꼭 다시 방문할 것 같습니다! 예쁜 잔도 따로 고를 수 있어서 너무 좋았어요!  고등어 봉초밥 
    안키모 오이김밥

    요녀석들 강추 임니당ㅋㅋㅋㅋ 혀를 재밌게 하는 녀석들이에요…. 고급지고 신선하고 맛있음.  
    담에 또먹을거에영 🥹  미슐랭 별 드립니다(?)  오랫만에 친구와 만나 소주 한잔에 맛난 음식을 먹습니다. 항상 좋은 곳.  코스 구성이 알차고 제철 재료를 잘 살린 플레이팅이 인상적이었어요.
    초밥은 밥 온도랑 생선 숙성이 잘 맞아서 전체적으로 밸런스가 좋았어요.
    디저트까지 퀄리티가 일정해서 끝까지 만족스러웠어요.
    셰프님 설명도 친절하고 진심이 느껴졌어요.
    재방문 의사 있어요 :)
    첫방문인데 분위기도 좋고 직원분들도 너무 친절하시네용
    숙성회랑 안키모 호소마끼 정말 맛있어요!
    데이트하기 좋고 소개팅으로 방문해도 너무 좋을 것 같아요
    즐거운 시간 보내다 가겠습니다아🩷💓💗💕 계속 와보고 싶었던 식당인데 드디어 오게 돼서 너무 좋은 시간 보내다가 갑니다~ 에피타이저로 나오는 토마토 디쉬도 너무 맛있었고, 사시미랑 아귀간 김밥도 너무 맛있었습니다 :) 
    다음에 오면 고등어 봉어초밥도 먹어보고 싶네요!! 봉초밥 맛있어요 회 아쉽고요 특히 참치 피맛 아쉬워요
    가격 많이 애매합니다 특히 사케 가격 애매해요
    회도 양이 많이 적어요 
    고등어 봉초밥 하나 보고 갈거면 가도 될것 같아요.
    의자 불편하고 가게도 좁았네요 분리된 룸이 있는 지 몰랐는데 조용하고 좋았어요! 사시미는 당연히 맛있고 고등어봉초밥이 완전 맛있었습니다! 다음에는 다른 메뉴도 먹어보고 싶어요! 워낙 인기있는곳이라 꼭한번 와보고싶었는데 추석 연휴에 미리 예약을 해두고 드디어 방문했습니다.
    고등어봉초밥 너무 맛있어서 1인 1접시 먹고싶을 정도네요... 또오고싶습니닷. 해방촌에서 제일 좋아하는 나의 아지트.💙
    모든메뉴가 다 맛있어요! 안키모 존맛
    음악도 좋고 술도 맛있고 안주도 맛있어요 !! 
    다음에 또 올게요 적당히 시끄럽고 분위기 좋아요 맛도 있어요
    메뉴 오래걸릴까봐 걱정했는데 순차적으로 주문하다보니 불편함 몰랐어요 대구에서 고등어봉초밥이 맛있다고 해서  왔는데,,, 입에서 살살 녹네요!  남자친구가 창가자리 미리 예약해서 왔는데 낭만있고 좋아요.ㅎㅁㅎ  사시미가 쫄깃쫄깃해서 맛있어요!
    안키모 호소마키도 고소해서 너무 맛있고 직원분들도 친절하세요!
    다음에도 또 방문하겠습니다 :) 3년전부터 오는 곳인데
    고등어봉초밥은 여기 이길 곳 없어요... jmt! 예약하고 홀테이블에 앉아서 식사를 진행했습니다. 조용한 음악과 조명이 너무 좋았습니다. 음식은 너무 맛있었는데요, 고등어 봉초밥 너무 추천합니다👍 2차로 왔는데  음식이 다 맛있어서 배부른데도 자꾸 먹게되네요 ㅜ 아귀간이 올라간 안키모 호소마끼  눈이 번쩍 뜨이는맛이에요 :) 너무 맛있어요!!스지 어묵탕은 조린 무기 예술 ㅜ 다음에 다른 메뉴 먹어보러 또와야겠어요 :) 음식 하나하나가 다 너무 맛있어요!

    분위기도 좋고 배부른데 계속 흡입했네요^^
    안키모호소마끼 2개 시킴!!ㅋ 분위기 좋고 사시미 너무너무 맛있어요 !!!!!!
    친구들이랑 잘 먹고 갑니다~ 고등어봉초밥 먹으러 재방문했는데 사시미도 너무 맛있어요! 술이 술술 들어가네용 ㅎㅎ
    일찍 예약해서그런지 룸으로 왔는데 분위기도 좋고 만족스러운 식사였습니다~
    고등어봉초밥은 여기가 제일 맛있어요 !! 친구 생일기념으로 왔는데 음식들이 다 너무 맛있어서 재방문하고 싶어요~ㅎㅎ
    특히 고등어초밥이 진짜 맛있어요!!ㅎㅎ 고등어봉초밥 맛있게 먹었던 기억이 있어서 재방문했습니다! 마라볶음면 안주로 너무 좋아요!
    잘 먹었습니당 고등어 봉초밥이 정말 놀라울 정도로 맛있었습니다. 신선한 재료에 밥의 간도 딱 맞아서 한입 먹을 때마다 감탄이 나왔습니다. 사시미도 신선하고 부드러워서 입안에서 사르르 녹는 느낌이었어요. 솔직히 음식 하나하나 전부 완벽했습니다. 분위기도 조용하고 세련되어서 특별한 날에 오기 정말 좋은 곳이었습니다. 여자친구 생일이라 방문했는데 여자친구가 너무 행복해해서 저까지 기분이 좋았어요. 덕분에 잊지 못할 하루가 되었습니다. 다음에 꼭 다시 방문할 생각입니다. 분위기가 좋고 직원들의 응대가 너무 친절해서 좋아요!
    에피타이저도 맛있었구 고등어 봉초밥이 진짜 맛있어요 강추합니다 ^^
    고등어 봉초밥 정말 맛있습니다.. 감동받았어요.
    부타가꾸니는 동파육급으로 부드러웠습니다. 너무 맛있고 분위기도 좋아요
    ☺️💓💓💓 아담하고 아늑한 분위기가 너무 좋습니다. 사시미가 입에서 녹아요. 자주 오게 될 것 같아요 사시미 도미솥밥 너무 훌륭해요~! 특히 사시미가 너무 신선하고 달아요 강추!! 음식이 전체적으로 아주 훌륭하고 분위기도 좋아서 대화하기 아주 편안하고 좋습니다 특히 안키모 후소마키 강력 추천드립니다 안시키면 손해 봉초밥 1티어집이라해서 왔는데 너무 맛있습니다....💓
    사시미도 쫀득쫀득 맛있습니다..
    맛있는 음식먹고 너무나 행복한 서울의 밤 입니다~~
    꼭 드세요 강추! 메뉴가 하나하나 너무 맛있고 비오는 날 해방촌 분위기도 너무 좋아요! 부타가꾸니, 안키모마끼는 꼭 드셔보세요!  단골이예요🩵 올 때마다 감동이예요🤩🤩 고등어 봉초밥 꼭 예약하고 드세요!!! 오마카세에서 먹던 것보다 더 맛있어요 🥹 주문해보고 싶은 메뉴가 너무 많아서 재방문의사 백퍼입니당 👍 예전부터 다니던 맛집!! 너무 맛있어요  맛있는 음식 덕분에 비가 오는 날씨에도 가분좋은 시간 보냈습니다. 음식이 맛있는 것 같아요. 캐치테이블 예약팀은 별도 공간으로 된 룸으로 자리 안내해주셨는데 완전 프라이빗한 룸은 또 아니라서 약간 애매했어요. 옆 테이블과 모든 대화를 공유하는 느낌 … 메뉴 하나하나 너무 맛있고 ,, 친절하시고 ,,
    오랜만에 친구들 모임으로 만난거였는데
    예약하고 칭찬 받았어요 ㅎ ㅎ vV 룸으로 예약을 했는데 정말 프라이빗하고 분위기도 너무 좋았어요! 음식과 술 조화가 잘 어울려가지고 다음에
    또 올게요  해방촌 오면서 꼭들려야할 식당이 있다고 들어 방문했습니다! 고등어봉초밥 꼭 드시길 추천드립니다. 비리지 않고 너무 맛있었어요!  봉초밥 기대 반 비릴까봐 걱정 반으로 왔는데 진짜 너무 최고였어요,,,
    너무나 행복 생일 보내고 갑니다
    감사합니다🩵 제 또또또간집인 심야식당 기억 ㅠ
    저만 알고 싶어서 리뷰를 남기고싶지 않지만,,
    남겨봅니당
    제가 먹어본 고등어 스시, 고등어 초밥 중에 단연 1등이고,
    진짜 맛있어요!! ㅎㅎ 가을이 오기만을 기다리다가 
    올해도 찾아왔습니다. 번창하시고 매년 찾아오겠습니다🖤 고등어 메뉴가 생각보다 약간 비린 것 빼고 전반적으로 나쁘지 않았습니다. 아직까지 숙성회는 일본이 훨씬 잘 하지만 한국에서 이 정도면 평균 이상은 하는 것 같기도 하고… 많은 숙성회들을 먹어봤지만 꽤 맛있네요! 특히,안키모 최고에요! 감사합니다! 고등어초밥을 처음 먹어봤는데 비린맛도 안나도 정말 맛있어요!!! 역시 메인메뉴입니닷. 하이볼도 맛잇고 전복 아란치니도 맛있었어오!!! 같이 먹으면 좋을 것 같네용! 매년 봉초밥먹으러 오는 곳이에요 ! 
    진짜 봉초밥은 여기보다 맛있는곳 없는거 같아요ㅠㅠ
    다른 음식도 다 맛있어요 ㅠㅠ최고👍👍 두번째 방문입니다 
    두번을 와도 두번 다 맛있는 적은 처음입니다
    까다로운 맛집감별사 피셜 가장 맛있는 서울 음식점
    고등어봉초밥은 가을에 꼭 드세요
    고등어 봉초밥을 꼭 드셔야합니다 고등어봉초밥 진짜 너무 맛있어서 먹기 아까워요…
    스지오뎅탕도 넘넘 맛있답니다..ㅠㅠ 에피타이저로 주신 토마토랑 고등어 봉초밥 진짜 너무 맛있었어요 초밥이 생각보다 커서 생각보다도 더 배부르게 먹었습니다.부타가쿠니도 오겹살이랑 포테이토랑 루꼴라 조합이 너무너무 맛있었어요 예약 못해서 봉초밥 못먹었으면 서운할뻔했습니당 꼭 예약하시고 방문 추천 고등어 봉초밥 진짜 맛있어요!! 사시미도 같이 시키길 잘 한 것 같아요~~테라스 자리에 앉았는데 바람 솔솔 들어와서 시원하고 분위기도 좋아요! 데이트 장소로 제격입니다~~ 좋은 분위기에 매우 퀄리티 있는 메뉴와 주류 

    다음에 또 방문하고 싶은 훌륭한 퀴진이에요~!💕🌙 고등어 봉초밥 맛있다는 얘기가 많아서 방문했는데 역시 맛있었습니다! 사시미도 전반적으로 훌륭했지만 특히 잿방어가 맛있었고 술과 함께 먹기 좋은 음식들이었습니다!
    적당한 소음에 분위기도 좋아서 데이트나 친구들과도 함께하기 좋은거 같습니다 예전부터 해방촌오면 와보고싶었던 곳인데, 음악도 넘 취향이구 음식도 하나같이 깔끔하고 맛있었어요. 특히 에피타이져로 나온 토마토&리코타도 유자향가득에 넘 기분좋게 저녁시사를 시작할 수있었슴니당💗  우나기덮밥이 진쨔 마싯서효,,, 매장 분위기도 너무 넓지않고 아늑해서 좋습니다 :) 오랜만에 맛집 찾은것 같아서 좋아요 또 올게요! 고등어 봉초밥은 신선하고 고소해서 꼭 맛봐야 할 시그니처 메뉴였어요. 삼치 솥밥은 밥알에 담백한 풍미와 대파 향이 잘 배어 있어 너무 만족스러웠고, 마라 볶음면은 자극적이지 않으면서 알싸한 맛이 인상적이었습니다. 전체적으로 식재료의 신선함과 퀄리티, 다양한 맛의 조화 덕분에 해방촌에서 특별한 한 끼를 즐길 수 있었습니다. 예약 필수, 인기 메뉴는 미리 주문하면 더 좋을 듯합니다. 고등어봉초밥이랑 장어덮밥 필수입니다! 고등어봉초밥, 부타가꾸니, 스지어묵탕, 가지라구 먹었는데 진짜 다 맛있어요. 분위기도 코지하고 저녁에 친목모임으로 강추입니다!!  친구들이랑 술한잔 하러 왔는데, 분위기도 좋고 음식도 너무 맛있어여! 봉초밥 꼭 드세여!!!!!! 분위기가 너무 좋고 데이트 하기에도 너무 좋을 것 같아오 오늘은 청모라서 왔는데 다음엔 애인이랑도 와보고싶어요! 친구들끼리 날씨 좋은날 분위기 내기에도 좋은 것 같아요!  생일 기념으로 왔는데 분위기 너무 좋고 대화하기에도 안성 맞춤이예요 ㅎㅎ 다들 고등어봉초밥 맛있다고 하네요 제주도민도 있는데 만족한대요 고등어봉초밥이 역시 맛있습니다! 부타가쿠니도 남자친구가 너무 맛있게 먹었어요. 다음에 또 올게요!
    분위기 좋고, 무엇보다 음식이 너무 맛있어요. 고등어 봉초밥 하나도 안비리고, 마라볶음면은 불닭맛이 딱인것 같습니다. 
    강추 드립니다!!!!  생일때마다 방문하는데 분위기도 좋고 음식도 맛있어요. 고등어초밥 완전 강추입니다. 안키모 완전 추천이에요! 매장도 좌석이 다양해서 예쁘고 분위기/맛 다 너무 좋아용♥️ 분위기도 좋고 음식이 맛있어요!
    고등어봉초밥도 예약하지 않았는데 친절하게 해주셨어요:)
    사케 잔도 다양하고 귀여워요~ 특별한 날 방문하기 너무 좋아요.
    사시미 하나하나 다 맛있고 고등어 봉초밥도 먹어본 곳 중 제일 맛났어요👍👍 고등어봉초밥 진짜 인생 초밥이에요 . . . ♥ 너무 맛있어요 ! 해방촌 처음인데 덕분에 좋은 기억으로 남을 것 같아요 ♥ 일행이 봉초밥 마니아인데 먹어본중 가장 맛있다고 합니다.
    분위기가 좋아서 연인이나 소개팅으로 오면 좋을것같아요! 진짜 서울 고등어봉초밥 1 등은 무조건 여기 입니다 
    근데 숙성회도 퀄리티 미쳤고 감칠맛 터져요 
    도미스테이크도 엄청 맛있고 
    직원분들 너무너무 친절하고 매장 분위기 환상이에요 
    얼른 다녀가세요  첫 방문이었는데 너무 맛있게 잘 먹었습니다!  너무 맛있어요!! 벌써 4번째 방문 자주 오는 곳이에요~!!! 변함없이 맛있고 분위기 좋은 곳 ❤️ 너어무 맛있어요! 담에 또 올거 같아요~ 솥밥 꼭 시키세요 너무 맛있어요 ㅠ  음식도 맛있고 깔끔한 분위기라 좋았어요. 다음에는 꼭 미리 예약해서 고등어봉초밥 먹어보고싶어요😄 와인도 칠링해 주셔서 맛있게 즐길 수 있었습니다.  메뉴 전부 다 너무너무 맛있고 분위기도 좋아요🤍🤍🤍 다음에 또 방문 할게요! 고등어봉초밥 꼭 드세여! 첫입 감동 심합니다. 먹다가 기절하고도 먹었어요🫠 정말 신경많이 써주시고 친절하세요! 서비스도 계속 주시고 다 넘 맛있어요. 사진엔 안찍혔는데 마라도미구이 진짜 맛있어요😭💟 음식도 맛있고 
    분위기도 좋았습니다 
    감사합니다! 분위기도 좋고 맛있어요 고등어봉초밥 먹고싶어서 미리 예약하고 방문했는데 진짜 맛있네요! 재방문 의사있습니다 추천해주신 사케도 좋았고 음식들도 너무 맛있었어요!
    사장님이 너무 친절하십니다!  조만간 또 방문하겠습니다~ 갈때 마다 너무 맛있어요,,,
    지인들이 맛집 추천해달라하면 첫번째로 말하는 식당입니다!! 직원분들 항상 친절하시고 음식 맛은 말해뭐해입니다,,, 특히 이 집 봉초밥은 먹을 때마다 박수치면서 먹어요!! 고등어는 추워지면 맛있다는데 이번 겨울에는 10번 먹는게 제 목표입니다ㅎㅎㅎㅎ 봉초밥 뿐만 아니라 사시미도 최고였고 부타가쿠니랑 안키모도 필수로 주문하셔야 합니다ㅎㅎ 무엇을 선택하든 다 맛있고 만족하는 집입니다!!! 고등어봉초밥 미쳤어요!!! 시그니처 메뉴인 이유가 있네요 너무 잘 먹었습니다. 모든 메뉴가 다 맛있었어요! :)  인생 고등어 초밥 맛집입니다.... 진심이에요 흑흑 🥹
    다른 메뉴도 좋은데 고등어봉초밥이 압도적이에요 시킨 메뉴 모두 다 너무 맛있었습니다!!
    사시미는 물론이고 봉초밥이 미쳤어요 예약하고 꼭 드시는걸 추천합니다!! 나중에 또 방문할게용😊 셰프님 저희 집에 모시고 가고 싶습니다…고등어 봉초밥 1티어 🐟 입에 넣자마자 사르르 녹고 먹을수록 더 맛있네요 ㅠ 이 집 신고할께요.
    오늘부터 나만의, 너만의, 그리고 우리의 [맛집신고]🥵🥵💦💦 오랜만에 이쪽 동네 방문해서 찾아왔어요 ㅎㅎ 고등어봉초밥 너무 좋아하는데 장인이라고 하셔서 기대하고 먹었던 게 딱 기대감 충족되고 너무 좋았네요 가게가 넓은 건 아니지만 분위기도 딱 좋고, 다른 대표메뉴인 것도 먹었는데 맛있었어요! 잘 먹었습니다~ 직원분들도 너무 친절하시고 음식은 진짜 말할 것도 없이 너무 맛있었어요! 해산물을 즐기는 편은 아닌데도 고등어봉초밥도 비리지않고 맛있었습니다! 다음에 또 올게요🩷 여기 진짜… 고등어 봉초밥 꼭 드시고… 솥밥도 꼭.. 라구도… 고기도… 진짜 제발제발제발데발 꼬고고고고고곡 드세요 ㅠㅜ 맨날 예약하규 재방문하고 있어요 ㅠㅠㅠ 비싸고 멀어도 이거 먹으러 해방촌 옵니다. 이전부터 정말 맛있어서 이번에는 외국인 친구들도 데리러 왔어요 ㅎㅎ 음식도 너무 맛있었고 덕분에 좋은 시간 보냈습니다~! 직원분들 섬세하시고 오랜만에 즐거운 식사가 됐습니다 코스처럼 먹었어요 진짜 여자친구가 회에 예민한데도 불구하고 또 오고 싶다는 곳이었구요. 예약 필수이니깐 꼭 예약하고 오세요. 논알콜도 맛있어요. 무조건 추천 음식드시구 솥밥도 필수입니다!!! 고등어 초밥 맛있어요 ㅎㅎ 분위기도 좋아요 커플끼리 오기 좋은것같아요~ 고등어 봉초밥 꼭 예약하세요 꼭입니다 5번째 방문 ㅎㅎ 항상 넘 맛있어요 모든 메뉴 다 맛있어요 외국인친구들도 입에서 녹는다고 너무 좋아하네요 ㅎㅎ 숙성사시미가 진짜 입에서 살살 녹아요ㅠㅠ 고등어초밥도 꼭 예약해서 드세요💙 분위기 너무 좋고 사시미도 너무 맛있어요~!
    고등어봉초밥은 필수!!! 으아 겁나 맛있습니다 가을에 고등어봉초밥 땡길때마다 오는데 올때마다 기대를 저버리지 않습니다. 고등어봉포밥 좋아하시는 분들은 꼭 오셔서 즐기세요! 지인들이랑 왔는데 
    음식들도 너무 예쁘게 나오고
    맛도 너무 너무 맛있어요 ! ❤️❤️❤️😍 친구 청첩장 모임 덕분에 처음 방문해봤는데 정말 너무 맛있어요! 많은 메뉴시켰는데 정말 다 만족입니다! 너무너무너무 맛잇어요
    안키모랑 고드어봉초밥 최애메뉴고
    아란치니도 너무 맛있어용! 자주 오는 집이에요! 오늘도 넘 맛있어요! 청모를 위해서 방문했는데, 너무 좋았어요! 
    음식도 맛있는데 예쁘기까지!! 
    다음에 또 올래요~~  친구들이랑 오랜만에 만나서 맛있는거 먹는데, 사시미 너무 맛있고 가게 분위기도 좋아요!! 고등어봉초밥 너무 맛있어요!! 하나도 안 비리고 입에서 살살 녹아요 ㅠㅠ 마라면이랑 봉초밥 조합도 너무 좋아요!! 항상 고등어 초밥 먹고싶을때마다 여기로 옵니다~!!! 고등어봉초밥 인생 고등어봉초밥이에요!!!!!
    한 접시 더 시키고싶은거 겨우 참았습니다 두번째방문이에요~ 진짜 나무 맛있어요
    지인들께 추천해주고 싶은 식당이에요🤤 무조건 가세요. 제발 가세요. 좋은 날 좋은 음식으로 하루를 완성 시키고 싶다면 그냥 가세요.

    오랜만에 먹었는데 더 맛있어졌습니다. 더 맛있어질 수 없을 것만 같던 봉초밥과 솥밥.. 이러니 계속해서 안올 수가 없네요. 심야식당 기억 때문에라도 열심히 돈 벌어야 겠습니다.

    감사히 잘 먹었습니다! 지금 벌써 몇번째 방문인지 모를정도로 왔는데…. 너무너무 맛있어요
    항상 둘이 와서 다 못먹더라도 메뉴 세개이상 먹구 가는거같아요 
    고등어봉초밥은 진짜 꼭 드세요!!!!!! 고등어봉초밥 미리 예약하고 갔는데 아주 맛있었습니당
    캐치테이블로 미리 예약하고 가는거 추천드립니다
    분위기도 좋고 유자사케도 맛있고, 망고 맥주도 맛있었습니다!! 사케 시키면 잔도 고를 수 있어서 좋았습니다 
    메뉴도 금방 나오고 다 맛있어서 다음에 또 올 것 같습니당 너무 맛있고 만족해요 지인들이 만족해사 좋아요 나는 이미 단골 ~ 
    장소,분위기,맛 다합격
    스지 어묵탕과 호소마끼 그리고 유자하이볼로 스따뜨!
    음식들이 전부 맛있어서 추가 할수밖에 없슴 
    이맘때 창가 자리는 명당이니 미리 예약하시길 ~ 고등어봉초밥이 진짜 맛있어용
    사시미도 맛있습니다!! 인기가 많아서 캐치테이블 예약을 하고 오는게 좋을 것 같아요
    음식도 빨리나오고 분위기도 좋습니당
    다음에 또 올게요! 룸으로 가서 조용하고 좋았어요 음식듀 맛있고 직원분들도 친절하십니다~ 3명이상은 룸으로 예약 추천드려요 사시미가 너무 맛있고 사케 콜키지 했는데 잔이 너무 예뻐요!
    직원분들도 굉장히 친절하시고 분위기도 좋습니다.👍
    고등어 봉초밥 꼬옥 추가하세요!!!!!!!!  분위기 좋고 음식도 너무맛있습니다 특히 고등어봉초밥!!!! 분위기 좋고 고등어 봉초밥은 무조건 예약해서 드시러 오세요. 직원분들도 너무 착하고 음식도 맛있어요!
    안키모 오이김밥 강추 입니다! 안키모 좋아하는데 진짜 무시무시하게 듬뿍 올라가있어서 넘 맘에 쏙 들었구 자리도 창가자리 넘 낭만있었어요 가끔 마을버스 창문으로 사람들과 아이컨택하지만ㅋㅋㅋㅋ그만큼 바깥이랑 가까운 자리라는 뜻!! 사시미도 맛있구 미리 예약해서 좋은 자리 앉으면 진짜 최고 하루 마무리인 곳🥰 주차가 힘들지만 맛있어요!! 한번갔다가 맛있어서 가족데려왔어요 바 자리라 분위기 좋았어요 ㅎㅎ 특히 고등어 봉초밥 정말 맛있습니다! 제 최애 음식점이에여
    매년 한번은 꼭 오려고 하는데
    고등어봉초밥은 진짜 넘버원이에요 고등어봉초밥 하낫도 안비리도 너어어무 맛잇어요💛
    고등어봉초밥 너무 먹고싶었는데 완전 만족입니다~
    부터가꾸니도 너무 부드럽고 매쉬드포테이토와 조화가 좋아용!
    고등어봉초밥은 리얼 레전드에요ㅠㅠㅜ 다른곳이랑 차원이 다름ㅠㅜㅜ 재방문의사 있어요 분위기도 너무 좋고 고등어봉초밥 진짜 맛있어요! 아내 생일로 왔는데 완전 만족입니다 고등어봉초밥은 예약을 해야만 먹을 수 있다길래 미리 예약을 했었는데요, 진짜 세상에서 제일 맛있는 음식을 꼽으라면 이 봉초밥을 고를 것 같습니다.
    그리고 기대만큼 맛있었던 안키모 산은 진짜 꼭 시키세요. 후회없는 선택입니다.
    비 내리는 저녁, 예쁜 공간에서 소중한 사람과 맛있는 음식을 먹으니 너무나도 행복합니당 감사합니다 짱~💗 오는데 차가 막혀서 예약간보다 많이 늦었는데 친절하게 맞아주셨습니다. 비오는날 창밖보며 술 기울이기 최적의 분위기입니다. 비가 와서 더 운치 있고 좋네요 숙성회는 비린내 전혀없고, 고등어봉초밥은 먹어 본 사람만 왜 미리주문해서 먹는지 알 수 있을 정도로 맛있습니다. 내돈주고 오기 너무 좋은 곳!!!! 후기 좋은 이유를 알거 같네요.  고등어봉초밥과 사시미 먹었는데 너무 맛있어요 하나도 안 비리고 술땡깁니다,, 노래나 전반적인 분위기와 어우러져서 만족100%입니다! 너무너무 맛있어요 분위기도 정말 좋구
    메뉴도 정말 맛있더라구요!
    특별한 날 추천두려용ㅎㅎ 진짜 맛있어요 ㅠㅠ 프라이빗룸도 조용하고 분위기 너무 좋아요!!! 최고!!  분위기도 좋고 음식도 너무 맛있어요!!
    회식때 마다 오는데, 공간도 프라이빗해서 너무 좋았습니다 안키모 김밥 미쳤어요 .. 너무 고소하고 오이랑 잘 어울려요 ..🫶 사시미는 말해뭐해입니당 두께도 적당하고 가격도 이정도면 괜찮은 것 같아요!! 지난번에 오고 너무 맛있어서 남자친구 데리고왔어요!
    고등어봉초밥 회나 초밥 잘 안먹는 저도 진짜 맛있게 먹었어요. 숙성회도 식감, 맛, 신선함 뭐 하나 빠지는거없이 맛있습니다. 
    선선한 바람드는 창가가까이 앉아서 분위기도 좋았고 즐거운 시간 보내고갑니다 회 맛있고 봉초밥은 말도 못합니다
    근데 산토리 생맥주는 10잔 드세요  결혼기념일기념으로 대구에서 서울로 남편과 딸과 여행왔는데 룸으로 안내해주셔서 편하게 잘 먹고 갑니다. 그리고 맛또한 너무 맛있었습니다! 재방문의사 너무 있는곳이었습니다.  고등어초밥 너무 맛있었어용! 다음에도 또 방문하고 싶습니다 ㅠㅠ! 음식이 전반적으로 맛있습니다.
    사시미도 맛있고 특히 솥밥 추천드립니다. 고등어봉초밥 비리지 않고 고소하고 맛있어요! 다음에 또 올게요~! 안키모는 나쁘지 않았으나
    숙성회는 숙성회 특유의 감칠맛이 떨어졌음
    비주얼은 좋았지만 숙성이 덜 된 느낌
    직원들이 메뉴 숙지가 덜 되어 안내가 부족했지만 불친절하진 않았음
    숙성회가 너무 아쉬웠음 기념일에 재방문 했어요! 고등어봉초밥 정말 맛있어요ㅎㅎ 다음에도 또 올래요 :) 사시미가 너무 신선하고 맛있어요🤍 고등어봉초밥이 진짜 또또또 먹고싶은 맛이에요,,,👀 스지어묵탕, 고등어봉초밥, 사시미 조합 최고에용🫢 정말 인생 고등어봉초밥 먹고 왔습니다!! 분위기도 좋고 너무 맛있네요! 먹는 내내 감탄했어요! 너무 맛있어서 3번째 방문입미다 ㅎㅎㅎ 다들 좋아해요 분위기도 좋구 진짜 너무 맛있어요ㅠ
    봉초밥과 부타가쿠니가 특히 G.O.A.T!
    매번 생각나서 고등어 기름이 올라올때면 항상 찾게됩니다! 작년생일에 방문하고 너무 좋은기억과
    맛이여서 이번에 생일이 같은 친구랑 왔는데
    고등어 봉초밥 미리 예약하고 다들 꼭 시키세요..!!
    가지라구도 제가 먹어본 라구 중 1등입니다
    진짜 다음에 또 오겠습니다 ㅎㅎ
    뭘시켜도 맛있어요 정말 다시 또 오고싶었던 ••• 봉초밥 사랑해요🥺🩷
    좋은 분위기 좋은 기억 행복한 추억 만들고갑니다!! 음식이 먹기 아까울 정도로 예쁘고 깔끔합니다
    맛있고 퀄리티 훌륭해요
    즐거운 시간 보내고 갑니다  안키모 호소마키, 아귀간이 엄청 고소하고 맛있어요. 고등어 봉초밥을 못먹어서 아쉬웠는데 다른 메뉴들도 엄청 맛있네요 분위기며, 서비스며 맛까지 최고입니다!! 지인 추천으로 가봤는데 맛있었어요 직원분들도 친절하세요 숙성회 맛있고요. 
    스지어묵탕도 참 좋아요. 
    분위기도 좋고 친절함이 더해지니 행복합니다. 
    또 올게요! 예약하고 왔더니 조용하고 분위기 좋은 공간으로 안내해주셨습니다!
    음식도 너무 맛있고 고등어봉초밥 먹으러 다시 오고 싶어요 :)
    다음에 또 방문하겠습니다! 도착하니 대기 16팀! 예약이 뿌듯했네요♡ 다 맛있었고 빨리 주셔서 좋았어요, 다만 고등어봉초밥 미리 예약 알림이 있었으면 더 좋았겠다 싶었어요ㅜ 예약 한 줄 알았는데 안돼있었어요 아쉽ㅜ 고등어 봉초밥이랑 어묵탕이랑 다 맛있었어용 ❤️ 너무 맛있어요~~ 신선해요!
    분위기도 너무 좋고~
    오랜만에 친구들이랑 힐링했어요!! 고등어봉초밥에 기대 많이 했는데 역시나 맛있었구 오뎅탕은 그냥 시켜봤는데 생각보다 되게 맛있어요!! 사케랑 잘 어울려서 좋아요 분위기도 좋고 데이트 장소로 추천합니다 ♡♡ 해방촌 맛집입니다. 음식도 맛있고 분위기가 미쳤어요. 고등어 초밥이랑 가지라구랑 오뎅 너무 맛있어요~~~ 친구들과 오랜만에 모임하려고 왔는데, 분위기도 좋고 너무 맛있네요~~~ 여자친구 생일을 위해 미리 찾아보고 예약했던 곳인데요. 여자친구도 저도 나오는 음식마다 만족도가 굉장히 높았고, 서비스도 너무 좋으셨어서 재방문의사 200%입니다! 고등어초밥이 정말정말 맛있어요!!  고등어봉초밥은 꼭 드세요 강추입니다. 다음에 꼭 재방문할께요 분위기도 너무 좋습니다. 친구 생일이라 예약해서 왔는대 직원분들도 친절하시고 음식도 맛있어요! 고등어 봉초밥은 예약해야하는데 맛있으니까 다들 꼭 예약해서 드세요~~! 벌써 네 번째 방문인데 역시 너무 맛있습니다 ㅎㅎ 맛있는 음식들과 포근한 분위기가 너무 좋았습니다~
    좋은사람들과 좋은 시간을 보내기 좋은 식당이었습니다. 저번에 좋았던 기억이 있어서 두번째 방문이예요~ 날씨 좋은날 창가쪽에 앉으면서, 맛있는 음식 먹으닌깐 스트레스도 풀리고 너무 좋아요! 몇번째 방문중인데 오늘도 너무 맛이 있네요🤗
    창가 앉아서 와인마시기 분위기도 좋아요🙌
    해방촌 대표 맛집이예용~ 시킨 메뉴들 하나 하나 정성스럽고, 최고였습니다.
    늦지 않은 시간 방문 했더니 한적해서
    좋은 시간 보내고 갈 수 있었습니다 :) 아버지랑 점심 먹으러 다녀왔어요!
    우드톤의 인테리어에 음악도 좋아서 편안하게 식사했습니다! 기념일 맞이해서 방문했는데 분위기도 좋고 음식이 너무 맛있어요 :)🙂
    고등어봉초밥 마라면 조합 너무 맛있어요! 맛, 분위기, 서비스 삼박자가 모두 완벽했던 저녁 시간이었어요👍🩷 모든 음식이 신선했고, 조미료 맛이 아닌 자연스러운 깊은 감칠맛이 느껴지는 음식들이 너무 최고였어요! 음식 먹는동안 잠시 일본에 다녀온 듯한 느낌이었습니다🥰 
    또 방문하고 싶어요!! 고등어봉초밥이랑 부타가꾸니 먹었는데 너무너무 맛있었어요~❤️ 오늘로써 2번째 방문입니다!! 고등어 봉초밥에 이끌려 예약을 하고 왔습니다. 별도로 지하1층에 있는 자리를 배정받았습니다. 봉초밥 스지 그리고 솥밥을 주문.. 둘이서 먹기엔 충분!!!! 담에 또 오겠습니다:) 고등어봉초밥 진짜 맛있고 가게 인테리어도 예뻐요. 다음에  다른 음식도 먹으러 또 오고 싶은곳이에요! 고등어봉초밥 먹으러 왔는데 다른 곳과 진짜 달라요..👍🏻👍🏻 왜 유명한 지 알 것 같아요. 사시미도 다양한 종류로 나와서 맛있게 잘 먹었어요!!  고등어 봉초밥 너무 맛잇어요, 제일 좋아하는 식당입니다. 오늘은 특별한 사람이랑 함께 왔는데, 좋아하네요~~
    자주 올게용❤️❤️ 분위기 좋고 음식 너무 맛있어요!!
    다음에 또 방문하고 싶습니다. 사시미도 너무 싱싱하고 다른 음식들도 너무 맛있었습니다. 봉초밥은 예약을 하지 않아 아쉽게도 먹지 못했지만 다음에 재방문해서 꼭 먹어보고 싶네요. 고등어봉초밥 꼭 시키세요!!!
    스지어묵탕도 너무 좋아요 스지추가 필수!!
    비오는날 친구랑 방문했는데 직원분들도 넘 친절하시구 
    사케 추천받아서 데워서 먹으니 꿀맛이네요
    재방문 할겁니다!! 예전에 와보고 분위기도 음식 맛도 넘 좋아서 또 왔어요!! 새로운 메뉴가 생겨서 먹어봤는데 고기가 정말 부드럽고 맛있어서 끝까지 맛있게 잘 먹었습니다!! 언제와도 변함없이 맛있고 분위기도 좋아서 다음에 또 오려구요😆 고등어봉초밥 왜 예약하는지 아는 맛이에요. 그리고 직원분들 모두 친절하시고 분위기도 좋아요. 하이볼이 맛있어요
    샤우 마라 면은 맛있는데 맵찔이먄 피하세요!! 조금 매워요
    숙성회가 입에서 살살 녹아요
    고등어봉초밥도 맛이 특이하고 맛있어요 맛있는 음식, 맛있는 와인 너무 좋습니다. 창가쪽에 앉았는데 분위기도 좋고 음식도 너무 맛있어요!
    고등어봉초밥 강추합니다👍🏻👍🏻 맛, 분위기, 직원분들의서비스와 친절, 모든게 완벽한 저녁이었어요!!!   헤방촌 로컬입니다. 중요한 손님이나 좋아하는 친구들이 해방촌에 놀러오면 1순위로 소개하는 식당이에요. 고등어 봉초밥은 제철이기도 해서 찾았는데 사실 올때마다 주문하게 돼요. 다른데서도 먹어봤는데 비리지 않고 너무 맛있어요. 기억 덕분에 입맛만 고급이 되는 것 같네요🥹🥹 메뉴 반은 다 먹은 것 같은데 친구들에게 맛집찾는 역할로 신뢰를 얻었습니다🥰🥰 해방촌 찐 맛집이니까 믿고 오셔도 될 것 같아요!!!  고등어봉초밥은 처음 먹어봤는데.. 와 이 맛있는 걸 이제서야 맛보다니 정말 미식적으로 좋은 경험이었습니다!! 🥰🥰🥰🥰🥰짱짱 고등어 초밥 미쳤습니다.. 인천에서 1시간 반 걸려서 왔는데 절대 후회 없는 맛입니다!! 강추~~ 진짜 1도 안비리고 고소한 풍미가 입안 전체를 감싸주는게 레전드네여. 재방문의사 있~~~~~~음 제가 먹어본 고등어 봉초밥 중 최고였습니다 ^^
    짱 맛있어용 ㅎㅎㅎ 음식이 다 맛있고 분위기도 정말 좋아요! 고등어봉초밥 정말 추천드려요!!  여자친구 생일이라 미리 예약해서 갔어요! 
    고등어 봉초밥이랑 부타가꾸니 주문해서 먹었는데 둘다 너무 부드럽고 봉초밥 비린내가 전혀 안 났어요!
    고등어 봉초밥 꼭 시키세요!! 고등어 마니아라 고등어요리 정말 많이 먹어봤는데요, 열손가락 안에 꼽히게 맛있네요 ㅎㅎ 같이 시킨 사시미도 전혀 비리지 않고 너무 맛있어요. 기념일로 간건데 정말 좋은 선택이었습니다🩷 고등어봉초밥 너무 맛있어요!!
    도미마라도 적당이 알싸하고 식감도 너무 좋아요
    추천합니다 데이트하기 좋아용 특히 맥주가 너무 맛있네요 😍 혼밥하기에 적적할 거라 생각했는데 쉐프님들과 직원분들과의 적절한 대화와 소통으로 소중한 시간을 보냈습니다. 너무 맛있어요~~ 
    분위기도 좋고 직원분들 너무 친절해요!

    고등어봉초밥이랑 매실하이볼 강추😋😋😋 제 생일 겸 방문했는데 모든 음식이 너뮤 맛있어요!!
    봉초밥 먹고 싶어왔는데 기름 좔좔 왜 예약 받는지 알겠어요 ㅠㅠ
    담에 남친 데리고 올게요!! 역시 언제나 뭘 먹어도 실패 없는 인생맛집❤️ 진짜 음식이 모두 맛있어요 !! 
    술 잔 고르는게 재밌구 너무 귀여워요! 음식은 맛있는데 직원(안경 쓴 사람) ㅆㅏ가지 없어요ㅠ
    1인 1음료 필수고, 가지 라구에 바게트 4조각 밖에 안나오고, 바게트 4조각 추가는 3,000원. 마라새우는 1단계(신라면맵기)가 신라면보다 훨씬 매워요.
    캐치테이블 리뷰 쓰면 연어초밥 서비스로 준다고 하는데, 4명이라 리뷰를 두 개나 쓰라네요? 예약자가 1명이라 다른 사람은 쓸 수가 없는데 융통성 1도 없고, 저 혼자 리뷰 썼다고 말해도 연어 초밥 주지도 않아서 못먹고 그냥 나왔습니다.
    테이블당 고등어봉초밥은 한 개밖에 못시키게 하면서 리뷰는 2개나 쓰라는게 어이없네요. 자리도 불편하고 춥고 마지막에 너무 불쾌해서 리뷰 씁니다. 모든 음식이 다 맛나요!
    원래 5시 타임 오고 싶었는데 직전에 예약하려면 인기 시간대는 예약이 어려워요!
    그래도 참으로 맛나요! 분위기도 좋고 일단 고등어봉초밥이 너무 맛잇어요
    안키호소마키도 너무 맛있네요.
    왜 달인맛집인지 알겠어요
    직원분들도 친절하고 맛있네요
    담에 또 올께요^^ 전반적으로 다 만족스러웠던 식사에요☺️ 음식도 맛있고 예약을 미리 해두니 따로 개별 룸으로 안내해주셔서 좋은 시간 보낼 수 있었습니다!
    점원 분들도 모두 친절하셨어요!! 진짜 봉초밥 좋아해서 이곳저곳 먹으러 다녔는데 생활의 달인에 나왔다고해서 이주전부터 예약해서 왔는데 분위기 안키모김밥 장어덮밥 봉초밥 전부 다 미쳤습니다 
    특히 잔 고를 수 있게 해주는 센스부터 최고에요!  음식이 너무 맛있고 , 친절하고 깔끔한 식당입니다.

    안키모도 너무 맛있어요!! 고등어봉초밥이 너무너무 맛있어요!! 완전 대만족스러워요~~ 고등어봉초밥 땡길때 또 올것 같아요ㅠㅠㅠㅠ 😍😍 고등어봉초밥이 유명하다던데 역시네여 ㅠㅠ 비린거 못먹는 친구도 맛있다며 다 먹었어요 ㅎㅎㅎ 분위기도좋고 맛있어요 😁 예약하고 와서 바로 들어왔습니다..! 메뉴들이 진짜 너무너무 맛있습니다🥺🥺 다음에도 또 방문하려고요 몇달전부터 저장해둔 곳인데 기대 이상으로 전 메뉴 다 맛있었어요!! 다음에 꼭 또 방문할 예정입니당🤍 고등어봉초밥 진짜 너무 맛있어요 꼭 드시구요 ㅎㅎ 전복 아란치니 그다음으로 추천!!!  고등어 봉초밥은 무조건 예약해서 먹어야 하는 메뉴입니다! 어딜가도 이맛이 안나요
    부타 가꾸니 사시미 모두 너무너무 맛있는데 오늘 처음 시켜본 안키모 호소 마끼도 진짜 맛있네요 메뉴다 다 사케랑 잘 어울려요
    분위기도 좋고 창가 자리에 앉았다면 지나가는 모든 사람의 부러움을 살수 있어요 확실히 맛있네요 ㅎㅎ 미리 예약해서 오길 잘했습니다. 
    다음에 또 올게요~ 고등어 봉초밥 하나도 안비리고 진짜 맛있어요!!!
    아귀간이랑 오이초밥개 맛있고 강추입니다~~!!! 
    예약하고 방문 추천합니다!! 모든메뉴가 너무 맛있고 내부 분위기가 너무 좋았어요!
    청모로 방문했는데 성공적이였습니당 ㅎㅅㅎ 고등어봉초밥 너무 맛있고요~ 회랑 다른것도 너무 맛있었어요~ 가게도 이뻐요 옆에있는 사람 기절시키고 혼자 다먹고 싶은 맛.. 지인추천받아 남편생일 기념으로 오게되었는데
    역시 고등어봉초밥 너무 최고입니다❤️💟💚🩶
    다음에 또 방문하고싶어요!!!!!🥳 음식이 맛있고 직원분들이 친절하세요! 벌써 두번째 방문입니다,
    같이 오는 지인들은 모두 분위기에 만족하고, 맛에 만족해요,^^ 무조건 실패없는 맛집 인정 입니당~~^^
    고등어 봉초밥이 진짜 맛도리⭐️⭐️⭐️⭐️입니다!!!!근데
    가지 라구도 맛있고 ! 새우 마라 벆음면도 징쨔 너무 마싰습니다 ☺️ㅎㅎ분위기도 맛도 너무 좋은 곳이네여 아지트로 저장 입니다  재방문인데도 맛도 분위기도 맛있어요ㅎㅎ
    안끼모호소마끼, 고등어봉초밥은 꼭 시키세요 !
    2명 3개 시키면 배불러용,, 또 오겠습니다 :-)  모든 음식이 다 맛도리에요~~~
    예약 꼭 하시고 고등어 봉초밥을 꼭 먹어보시길 :) 메뉴구성이 너무 알차고 회가 신선해서 좋았어요!
    분위기도 좋구 직원분들도 너무 친절 하셔서 좋은시간 보내다 갑니다❤️ 벼르고 벼르다 왔는데 역시 너무 좋아여 ㅠㅠㅠ 꼭꼭 예약하구 봉초밥 다들 드셔주세요ㅠㅠ 날씨 좋으니까 창문도 열어주셨는데 가을 크~~ 맛, 분위기, 서비스 다 대대 만족입니당♡♡♡♡ 분위기 너무 좋고 고등어봉초밥 ㅠㅠㅠ 진짜너무맛있어용 또오고싶은 맛집이네요🤍🤍 음식도 맛있고 분위기도 좋습니다. 감사합니다! 미리 예약한 고등어 봉초밥과 새우 마라 볶음면을 시켰어요. 에피타이저로 내어주시는 토마토 크림치즈 부터 시작해 맛이 너무 좋아 술집에서 식사를 한 바탕 하고 갑니다 ㅎ ㅎ 친절하셔서 더 좋아요 🤍 룸으로 예약했는데, 홀보다 조용하고 역시나 너무 맛있습니다!👍🏻 특히 고등어봉초밥 무조건 예약하세요~

    음식 먹는 순서는 숙성회 > 고등어봉초밥 > 부타가꾸니 > 스지어묵탕로 드시면 4명이서 배부르게 먹을 수 있는 것 같아요! 다음에 또 오기로 했어요!🙂 오늘 시킨 모든 음식들이 롼벽해써요!!!!! 솥밥도 안키모도 최고입니당..❣️ 분위기도 좋아서 데이트핳때 또올게용 분위기좋고 음식도 너무너무 맛있어요🪽 기대했던 고등어 봉초밥과 가지라구까지 맛있게 먹었어요!
    룸으로 안내해주셔서, 조용하고 분위기있게 즐겼습니다!!!

    재방문의사 100%에요 친구들이랑 방문했는데 분위기도 너무 좋고 맛있네요! 두번째 방문인데 언제와도 맛있고 분위기 좋은 맛집이라 너무너무 좋은 경험이었습니다 ◡̈ 
    예약없이 왔는데 다행히 워크인 가능해서 너무 좋았지만 예약하고 오시는걸 추천드랴요 !!
    사시미랑 돼지고기요리 너무너무 맛있었어요..🥹 생선의 퀄리티가 너무 좋았고 한 점 한점에 정성이 느껴졌던 것 같습니다! 고등어봉초밥을 여기서 처음 먹어봤는데 앞으로 다른 곳과 이곳을 비교하게 될 것 같아 두렵네요. 최고에용! 제 생일과 여자친구와의 기념일해서 두번째 방문인데 전 메뉴 거의 다 먹어본 결과
    그냥 맛없는 메뉴가 없습니다^^
    꼭 오셔서 드셔보시기를
    기대했던만큼 분위기도 좋고, 깔끔하고 맛있어요👍🏻 시그니쳐 고등어봉초밥은 최고였고 스지어묵탕도 너무맛있어서 국물까지 싹싹비웠어요!  숙성회와 함께마신 화이트와인 피노그리랑도 찰떡궁합 이었어요.
    고등어초밥이 하나씩 없어지는게 너무 아쉬웠던..😂 창가 바자리에서 만족스러운 저녁식사였어요!
    솥밥도 먹어보고싶었는데 너어무 배가불러서 다음 방문을 기약하겠습니다😋 룸이 있어서 좋고 고등어 봉초밥 정말 맛있어요! 음식이 다 맛깔났습니다!!!
    간도 적당하고 하나하나 다 너무 맛있었어요!
    강추입니다!!!!!
    음식 많이 시켰는데 맛있어서 다 먹어 빈접시만 남음요 고등어봉초밥 먹으러왔어요! 오마카세급의 맛이 납니다ㅠ안키모도 양이 많아서 기분이 좋아요
    또 올게요!!!!! 분위기와 인테리어가 너무 좋아요
    음식도 너무 맛있고 특히 고등어 봉초밥은 
    전혀 비리지 않고 맛있어요!
    재방문 의사 100%로 입니다
    음식도 너무 맛있고 직원 분들도 친절합니다. 또 방문할게요! 음식이 너무 맛있고 직원모두 너무 친절합니다. 좋은 음식덕에 즐거운 자리가 되었습니다^^
    숙성 사시미, 부타가꾸니가 정말 맛있습니다!! 안키모와 오이마끼도 진짜 일품입니다! 친구들과 맛있는 음식먹으며 분위기 내기 정말 좋은 곳입니다! 예약 못해서 고등어 봉초밥을 못먹은게 아쉽습니다!  모든 디시가 다 너무너무너무 맛있었어요 ... 🤍 친구랑 둘이서 안성재 쉐프님 빙의해서 칭찬 한가득하면서 황홀한 시간 보냈습니다아 ... 🫧🤤 다음에 또 올게요!  해방촌 왔다가 급하게 예약하고 방문하게 됐는데
    왜 유명한지 알 것 같아요..
    고등어봉초밥 비린 맛 하나도 안 나고 너무 맛있어요 ㅠㅠ
    부타가쿠니는 그냥 입에서 녹네요🥹 친구랑 생일겸 왔는데 분위기도 좋고 넘 맛있어요🤤 오이김밥은 기대 안 했는데 먹자마다 휘둥그레 했답니다 ㅜㅜ 고기는 그냥 입에서 녹아서 없어져요 ㅠㅠㅠ 최고 !! 고등어봉초밥은 꼭 예약에 고등어라고 적으세요 여러뷴..🥺 여러번 방문하지만 방문 할 때마다 좋은 기억 가지고 갑니다. 음식도 맛있고 뷴위기도 정말 좋습니다.
    특히 모둠 사시미와 고등어 봉초밥은 정말 맛있어요!!. 사이드 다른 메뉴들도 맛있으니 다 경험해보세요~ 처음 먹어본 고등어 봉초밥 정말 맛있네요
    직원분들 너무 친절하시고요
    맥주도 신선하고 맛있습니다!^^ 대애애애애존맛,,,하,,,벌써 3번째 방문입니다 이가격에 이 퀄리티 말이안돼요 숙성회 일단 무조건이고 어묵탕도 일본식으로 넘 맛잇어요 일단 뷴위기도 넘 좋아서 연인끼리 와도 좋고 걍 대강추입니다 숙성회 드실때 소금 추가 대추천 하 감사합니다 심야심당기억❤️ 너무너무 맛있고 오늘 하루 너무 최고로 만들어주는 맛입니다!!!너무 맛있습니다!!! 음식들이 다 너무 맛있어요 🤍🤍 
    분위기도 좋고 조도도 적당하네요 ✨ 벌써 3번째 방문이에요 ㅎㅎㅎ 고등어봉초밥 너무 맛있습니다! 완전 강추 드디어 방문하게 된 심야식당 기억, 역시…예약해서 올만했다..ㅎㅎ 고등어 봉초밥 진짜 1등, 제가 먹어본 것 중 제일 맛있었어요! 비린맛 하나도 없고 너무 맛있었어요 !
    마라 볶음면도 적당히 맵고 와인이랑 마시기도 좋았어요!
    다음에 생각날 봉초밥…또 올 생각입니다 ㅎㅎ 서울 최고의 봉초밥 맛집💛 같이 온 친구도 너무 맛있어해서 잘 먹고 갑니다!!! 다른 음식도 하나같이 다 맛있어요. 너무 맛있었어요. 식당 분위기 좋고 친절한 서비스 훌륭한 맛 👍 또 오고 싶어요!  봉초밥의 달인 맞습니다…너무 맛있어요ㅎㅎ 재방문의사 있어요! 직원분들도 너무너무 친절하시고 매장 분위기랑 음식 모든게 완벽했어요~!! 웰컴푸드도 맛있고 특히 고등어봉초밥 ⭐️⭐️⭐️⭐️⭐️ 지금까지 먹어본 고등어 봉초밥 중에 최고였습니다!!! 솔직히 웨이팅을 길게 했는데
    제가 부산 사람인걸 잊을 정도 였습니다...
    같이 온 친구에게 저의 식당을 알려주는게 너무 감사했습니다.
    너무 힐링하고 갑니다.. 분위기 좋고 데이트 하기 좋아요
    모둠사시미 비주얼도 예쁘고 맛있어서 만족스러워요 
    고등어 봉초밥도 미리 예약하고 시켰는데 
    안시켰으면 후회할 뻔 했습니당~~ 대만족!
    잘 먹었습니당 분위기도 좋고
    직원분들 친절하시고
    더블어 음식또한 맛있네요 

    멀리 지방에서 올라온 보람이 있네요 생일로 예약해서 왔는데요
    일단 인테리어가 매력있어요
    미리 예약했던 고등어 봉초밥은 명성대로 맛있었어요
    아귀간은 처음먹는데 오이랑 조화로워서 맛있더라구요
    좋은 식당 알게 되어서 좋습니다
    다음에 오면 다른메뉴들도 먹어볼거에요 메뉴 하나하나 다 맛있어서 데이트하기 너무 좋아요! 공간도 예쁘고 맛도 있어서 추천합니다 고등어 봉초밥 , 모둠사시미 존맛입니다💚💚💚💚💚💚 생각보다 조용하고 음식이 너무 정갈하고 맛있어요~~  고등어는 비린내가 단하나도 없네요~~ 친절하시고 가게가 예뻐요! 사시미를 제외한 거의 전 메뉴를 시켜보자! 라는 마음으로 시켰어요 ㅎㅎㅎ 처음 방문인데 전 메뉴가 다 맛있었어요! 재료를 아끼지 않고 막 넣어 주신것도 너무 좋았고~ 오랜만에 야간 시끌시끌한(?)분위기에서 있었던 것도 좋았습니다  기념일에 너무 맛있게 먹고 가요ㅎ.ㅎ
    음식, 분위기 모두 훌륭해요!!! 마구마구 소개하고 싶은 곳입니다:) 어느 하나 뺄 거 없이 다 맛있어요. 안키모호소마키도 고등어봉초밥 만큼 맛있어요. 어느 정도 먹은 상태에서 추가하는 마라새우면도 최고..ㅎㅎㅎ 심야식당 기억 외않가요..? 안키모 오이 꼭 시키세요!!!! 안에 분위기도 좋아요 두번째 방문임니당!
    고등어 봉초밥을 예약 못햐서 방문햤어요
    다음에도 올래요ㅠ
    너무 마싯어요ㅠㅠㅠㅠㅠㅠ 맛있고 생각보다 푸짐해서 배불러요

    나중에 술에다가 안주로 먹어도 좋을거 같아요 음식도 맛있고 분위기도 좋고 모두 만족스러워요! 재방문 할게요! 예약하고 방문했는데 개별룸으로 안내받았어요.
    봉초밥 강추드립니다 비린내도 안나고 너무 맛있었어요 강추드립니다!!!!!! 두번째 방문인데 구성이 너무 좋고 신선하고 맛있어용!! 고등어봉초밥 맛집에 왔는데 다른메뉴들도 대박이에요 특히 솥밥 기대안했는데 줜맛.. 술을 못마시니 아쉬워서 다음엔 술이랑 페어링해보려구요 부드러운삼겹살에 매시포테이토와 느키함을잡아주는 루꼴라 조합 너무 맛있어요. .
    고등어초밥 생전 처음 먹어보는데 입에서 살살녹고 비린내가 하나도 안나네요. .
    잠시 제로페이 안되는건 아쉽지만 재방문각입니다. .쵝오쵝오~~^^

    첫방문으로 생일파티를 하러 방문했는데 공간은 협소했지만 붐비는 느낌 없이 음식이 너무너무 맛있었어요 특히 고등어초바~~~~압~~!!❣️

    직원분들도 왕친절하시고 다음 재방문의사 200%입니다!!! 고등어 봉초밥 기대했는데 기대한만큼 맛있어요!!
    그리고 가지 라구도 진짜 맛있습니다 나중에 와서 다른 메뉴도 다 먹어보고 싶어요!! 음식도 너무 다 맛있고 
    직원분들이 모두 친절해요 
    분위기도 좋아요 
    재방문의사 100%입니다  맛있어서 재방문해요. 담에도 또 올게여 근데 가격은 좀 비싸여 예전부터 자주 오던 봉초밥 집이에요! 
    예약도 잘 안하는 XXXP인데, 여기 올 땐 꼭 예약합니다!!! 사시미 숙성이 진짜 잘 됐어요. 
    고등어 봉초밥은 꼭 드세요!!!
    금태 철일 때 솥밥도 먹어보고 싶네요
    가격이 아깝지 않은 맛집👍👍 음식 하나하나 다 맛있고 분위기도 좋아요
    화장실도 깔끔하고 좋습니다. 
    친구들이랑 약속 잡기도 좋고 데이트하기도 좋아요👍 회와 오뎅탕 먹었는데 너무 맛있어서 잘 즐겼습니다 :) 고등어봉초밥 그냥 미쳤어요ㅠㅠ 한 테이블 당 하나밖에 주문 안되는게 넘 아쉬워요
    스시 퀄리티도 먹어본 것 중 제일 좋았습니다 또 올게요!!! 원래도 애정하는 식당이었고 기념일에 늘 예약하고 방문합니다. 드셔보시면 알겠지만 모든 안주의 수준이 상당히 높고 특히 고등어봉초밥은 어나더레벨입니다. 거의 오픈하자마자부터 다녓어요... 열이먼 열 데려오는 사람마다 다 마싯다고 하고 ㅠㅠ 여기서 처음 고등어봉초밥이랑 안키모 머것는데 너무너무 마싯어서 중요한 일 잇을때마다 와요ㅠㅠ 하나도 안비리고 하나하나 퀄리티가 넘 조음 유일한 단점은 가게가 크지 않다는것...? 또 올게요 ㅠㅠ 봉초밥은 기대한만큼 맛있었고, 숙성회가 진짜 맛있어요! 분위기도 좋아요~ 고등어 봉초밥 예약해야 먹을 수 있어서 예약해 봤는데 ㅜㅜ정말 기대 이상이에요! 이거 보시는 분들 무조건 이 메뉴는 드세요,, 안키모도 살살 녹습니다 🫶 모든 메뉴 다 도장깨기하고싶은 곳!!! 사장님과 직원분들도 모두 친절하세요 또 방문할게요! 내부도 너무 예쁘고 고등어 초밥 정말 너무 맛있어요! 안키모 초밥도 정말 맛있었습니다 :) 고등어 봉초밥이랑 숙성회가 너무 맛있었어요 ! 분위기가 너무 좋아요~ㅠㅠ잔 너무 귀여워서 술맛 나고 고등어 초밥? 나오자마자 분위기 기절ㅠㅠ 오뎅바 안가도 되고 걍 여기서 조용하고 깔끔하게 먹을 수 있어요 제발 오시길,, 고등어 봉초밥 진짜 맛있고 분위기도 너무 예뻐요 ... ♡  일단 너무 청결해요. 그리고 정갈해요.
    음식이 맛있어여. 사시미 1인분도 가능해서 다른 음식과 같이 주문하기 좋아요.
    추천합니다.
    너무 맛있고 분위기도 좋고 서비스로 주신 에피타이저 너무 귀엽고 좋아요~!!! 최고의 식당 ~!! 또 오고 싶습니다 하하하❤️‍🔥💚💚💚👍🏼 봉초밥 너무 맛있고 분위기도 좋아요! 지인 모임 왔는데 음식 최고👍 
    분위기 최고👍 
    다음에 또 올겁니다 남자친구랑 기념일 맞이해서 왔는데 분위기도 너무 좋고 직원분들도 친절하세요☺️
    여기 오시면 고등어봉초밥은 꼭 드세요..! 진짜 맛있습니다🤤
    친구랑 애인이랑 상관없이 방문하기 좋은 곳이에요! 추천합니당🤍👍🏻 고등어 봉초밥 넘 마싯어용
    맥주 가격이 사악하지만 안주가 맛있어서 안시킬수 없는..😀 엄지를 계속 올리게 되는 고등어초밥입니다. 보통 기대가 크면 실망이 있을 법한데 왕 맛있습니다…  고등어초밥에 김 올려 먹으면 진짜 저 세상 맛입니다. 직원 분들도 친절하시고 생일 기념이어서인지 룸으로 받아서 조용히 즐겼습니다. 최고오👍 모든 음식이 너무 맛있고 분위기도 너무 좋아요!! 강추입니다!! 오랜만에 친구들과 모임 가졌는데 분위기도 좋고 음식도 좋고~ 너무 만족스러웠어요! 안주가 전반적으로 비싼감이 있는데 막 그렇게 맛있지도 않은 ,, 조금 애매했어요 ㅠ 와인도 전체적인 가격대가 높았네요 고등어봉초밥이랑 새우마라면 맛있어요!!ㅎㅎ 먹었던 음식들이 너무너무 맛있어서 너무 만족했습니다.
    봉초밥이 예약으로만 가능해서 한번만 먹고 또 못먹은게 너무 아쉽지만 다른음식도 충분히 너무 맛있어요 시켰던 음식 전부 맛있었어요~ 솥밥이 간도 적당하고 특히 만족스럽습니다. 쓰려고 생각하고 온 서울페이가 안돼서 아쉬웠지만 음식 맛, 분위기, 서비스 모두 마음에 드는 곳이에요. 정성스럽고 맛있는 메뉴들입니다 재방문 의사 있음! 너무 맛있어요 친구랑왓는데 또오려고여ㅎㅎ ㅜㅜ  넘 맛잇고 친절하거 분위기좋아요 또오고싶습니다 다음엥 미리 예약해서 봉초밥먹어보규싶어용 두번째 방문이에요!! 너무 맛있어요 ㅠㅠ 최애 고등어봉초밥 맛집 .. 개별룸으로 미리 예약해서 룸에서 먹었어요!!
    엄청 조용해서 모임하기 좋고, 음식도 정말 맛있어요!! 
    오랜만에 만나는 사람들과 분위기 좋게 즐겼습니다 고등어봉초밥이 레전드로 맛있어요. 마라 볶음면도 마라샹궈 같고 맛있어요. 가게가 아담한데 분위기가 좋고 데이트로 딱이에요 고등어봉초밥 너무 맛있어요! 마라 볶음면도 적당히 매콤하고 맛있어요. 가게도 아담하고 분위기 좋아요~ 독립적인 룸에 고등어초밥과 맛있는 안주가 술과 어울리고 분위기도 좋아요 분위기 정말 좋고 사진을 막 찍어도 그냥 인스타 감성 나와버리는 오랜만에 기분좋은 식당을 찾았습니다. 몇 번이고 더 올 것 같아요 ❤️ 아담하고 아기자기한 분위기예요
    고등어봉초밥 너무 맛있음!!!!! 고등어초밥 꼭 예약하세요. 제가 먹어본 고등어 초밥 중에 제일 맛있어요!!!! 가게 분위기도 좋고 술 종류도 진짜 다양해요!! 두명이서 오기 좋은 가게에요 2번째 방문입니다.
    솥밥도 맛있었는데, 인원수가 적어서 더못시켰어요.
    메뉴가 하나하나 너무 맛있어요
    몇번이고 재방문의사 있습니다 직원분들도 너무 친절하시고, 고등어 봉초밥도 너무 맛있어요 .. 룸이라 프라이빗하구 친구랑 너무 행복하고 좋은 추억 남기구 남기구 갑니다..💗 비린 냄새에 엄청 예민한데 사시미 하나도 안비리고 
    부위 마다 관리를 잘하신게 느껴집니다! 
    다음에는 고등어봉초밥 먹으러 올게요! 우연히 방문했는데 분위기도 좋고 회도 너무 맛있어요~~
    서비스도 주시고 감사합니다!! 잘 먹었어요🤤🩷🩵💛
    다음에도 올 것 같은 음식 맛집 분위기 맛집!!!!  기대했던 봉초밥은 못먹었지만 대신 다른 맛있는
    메뉴가 있다는걸 알게되서 더 좋았습니다

    다음에는 미리 예약하고 봉초밥 먹으러 와야겠아요~ 항상 고등어봉초밥 먹으러 옵니다❤️ 데려오는 사람마다 다들 맛있다고 극찬을 해요 오래오래 방문하고 싶은 가게입니다🩷 너무 맛있게 잘먹었습니다
    조용하고 오붓하게 데이트하기 좋아요 음식 하나하나 다 신선하고 맛있어요!
    분위기도 좋고 데이트 장소로 강추에요! 안키모 양이 넉넉하게 올라가있고 맛이 진해서 좋아요! 솥밥도 간이 적당하고 맛있네요 가게도 너무 시끄럽지않아서 좋았어요 :) 고등어 초밥 두번째 경험인데 비리지도 않고 맛있아요ㅎㅎ그리고 직원들도 친절해서 방문 추천드려용 고등어봉초밥 맛집이라고해서 예약하고 방문했어요ㅎㅎ
    서울에서 일본을 느낄 수 있어서 너무 좋았어요!
    다음에 또 올게요ㅎㅎ 음식 모두 너무 맛있고 다른 곳에서 쉽게 먹을 수 없는 메뉴들이라 더 방문할만한 가치가 있는 것 같아요👍 저번에 방문해보고 맛있어서 친구랑 재방문했어요 사시미도 맛있고 안키모랑 가지라구 다 강추드려요 분위기도 좋아서 데이트 장소로 추천입니다!  고등어 봉초밥 너무 맛있어요
    부산사람인데 회가 너무 맛있어요
    서울에서 먹은 회 중에 젤 맛있엉ㅅ 진짜진짜마싯어요 
    숙성회 대한민국에서 재일 맛있는 곳이에요🥰 두번째 방문인데 너무너무 맛있어요 
    친절하시고 매장도 쾌적하고 좋아요 🥸 해방촌 맛집 찾다 우연히 캐치테이블 통해 예약하게 되었는데  주문한 음식이 너무너무 맛있어서 대만족입니당☺️ 특히 고등어봉초밥 진짜 깔!끔! 봉초밥으로 유명하신줄 모르고 예약했던건데 너무너무 맛있네요ㅜㅠㅠ 다른메뉴 먹으러 또또또 오고싶어요🤭  진짜 너무 맛있네요.. 
    지금 메뉴 하나씩 다 시키고 있어요
    -캐치테이블 vip 였던 사람- 하나하나 다 맛있네요,, 메뉴판 다 시키고 가야 성이 풀릴 것 같습니다. ㅠㅠ 다음에 또 올게요ㅠㅠ 친구들이랑 맛있게 먹고 갑니다!! 고등어봉초밥이 맛있어요!! 안키모 오이김밥도 맛있어요. 다른 것도 먹어보고 싶습니다. 가게 분위기도 좋아요 생일 기념으로 친구들이랑 왔는데 프라이빗한 룸 너무너무 좋구 주문한 메뉴도 다 맛있어요 ㅎㅎ 특히 마라새우면 , 라구가지 넘 맛있…💗 고등어도 돼지고기도 모두 너무나 부드러웠다.
    기본적으로 향이 좋다. 루꼴라 샐러드의 향이 입 안에 남는다.
    고등어의 불향도 좋다. 늘상 최고에용.. 집 가까운 이자카야를 방문 해봐도 항상 가고싶다 생각 드는 곳은 심야식당기억 뿐인듯 합니다. 친구의 추천으로 방문햇는데 기대 이상으로 분위기도 좋고 음식도 맛잇습니다 :) 이태원 맛집 강추! 숙성회는 훌륭했고, 바게트랑 같에 먹는 가지라구도 맛있습니다. 함께 온 일행들도 다 만족했어요~ 첫방문인데, 
    룸도 프라이빗해서 좋았고,
    메뉴마다 다 맛있어서 정복하고싶어요ㅋ
    요청사항 잘들어주시고, 친절해서
    재방문의사100%입니다~ 제주도보다 고등어가 신선하고 맛있어요 ㅋㅋㅋ특이하게 깨를 주는데 잘어울립니다. 다른 메뉴들도 너무 맛있고 전부 간이 적당해서 부담없이 먹게 됩니다 음식도 너무 맛있고 직원분들도 친절하고 분위기도 좋아요 ㅎㅎ 덕분에 즐거운 서울여행 마무리 했습니다!! 고등어 봉초밥 진짜 맛있어요 꼭 예약해서 드세여! 가지라구도 인생 라구에용 b 예전에 친구랑 방문했다가 맛있어서 다시 방문했습니다. 이번에는 고등어봉초밥, 안키모 호소마끼, 쉬림프 마라볶음면을 주문해서 먹어봤는데 너무 맛있게 잘 먹었습니다! 분위기도 좋고 다음에 또 방문할 의향이 있습니다~! 친구가 맛있다고 추천해줘서 왔는데 고등어 봉초밥 꼭 드세요 !! 예약해야 먹을 수 있는데 진짜 맛있어요:) 전복 아란치니랑 호소마끼도 추천 !! 분위기 너무 느좋이구요~ 음식 진짜 다 맛있구요~ 
    매실 에이드도 맛있어요!! 음식들이 전부 맛있는데 그 중에서도 고등어봉초밥은 정말 맛있네요! 매실하이볼은 별기대안하고 마셨는데 새롭고 맛있었습니다 ㅎㅎ🫶🏻 음식 짱맛있어요 ㅠㅠ 고등어봉초밥 꼭 시키시길 ... 분위기도 좋아서 친구 혹은 연인끼리 와도 좋을 거 같아요! 오랜만에 만난 친구랑 같이 방문했어요! 회 신선하고 좋았습니다 ㅎㅎ  입에서 살살 녹아요
    진짜 맛있습니다
    제 최애픽은 마라도미입니다
    고소하고 넘맵지않고 고수향도 진하지않고 은은하고 크리스피마라까지 최고에요 완전 강추! 음식도 최고 음악도 분위기도 좋고 친구들과 좋은 추억 쌓고 갑니다. 친구추천으로 어렵게 왔는데 너무너무 맛있어요!!!!!!! 
    하룰라라갑니다🤤🤤 숙성회 진짜 너무 맛있고 분위기도 엄청 좋아요! 봉초밥은 꼭 예약하세요! 봉초밥이 이 곳 핵심이에요🩵
    부타가쿠니도 매쉬 포테이토가 킥입니다! 고등어 봉초밥 너무 맛있어요!!
    모든 메뉴들 다 맛있지만 봉초밥은 정말 짱 ✨
    맛있게 잘먹었습니다 ~ ^.^ ㅈ분위기도 좋고 주문한 음식들도 다 맛있었습니다!!  두번째 방문입니당♥️♥️
    고등어봉초밥 다시 먹었고, 숙성회도 비린 맛 없이 너무 맛있어요 
    다시 또 오고 싶어요 
    와.. 진짜 가지라구랑 사시미랑 솥밥 미쳤어요.. 입에서 살살 녹음.. 바게트 무조건적으로 추가..필요.. 음식 미쳤어요. 그냥 다 먹고 싶어요 두번째 방문이에요 :)
    고등어봉초밥 너무 맛있어요!!!
    또 오고 싶어용 잘 먹었습니다 🩵 분위기 좋고 고등어 초밥 미쳤어요 짱짱 맛있아요 추천받아서 왔는데 분위기도 좋고 맛있어요!! 다음에 또 올게요 모든 음식이 다 맛있어요.
    꼭 재방문 하겠습니다!! 고등어봉초밥 너무 맛있어요. 
    걍 다 맛있어요 굿 고등어봉초밥이랑 오뎅탕 시켜서 사케랑 같이 먹었는데 진짜 맛있고 술이랑 안주가 잘 어울려요! 고등어봉초밥은 진짜 예약해서 꼭 드세요!! 창가자리 사진보다 훨씬 분위기 좋고 친구랑 오기도 데이트하기도 정말 좋을 것 같아요! 다음에 또 고등어 먹으러 올게요☺️ 봉초밥 맛집찾아왔는데 너무 맛있고
    자리도 창가자리가 마음에 들어요
    또 방문할게요:) 다섯명에서 가니까 룸에 앉아서 편하게 대화할 수 있었어요 
    고등어초밥 너무 맛있습니다! 서울 놀러와서 저번에 힌번 와보고 너무 반해서
    또 서울 와서 들렸어요 짱맛! 음식이 전반적으로 깔끔하고
    가게 분위기, 직원분들 다들 너무 좋아요👍👍 대박 맛있어유 …
    안키모 진짜 강추 !!!! 세번은 더 먹을 수 있을것같아요 ..💛
    안키모는 진짜 필수필수
    저희는 고등어 봉초밥도 예얃해서 먹었는데요
    너므 맛있어요 …… ㅠㅠㅠㅠㅠㅠㅠㅠㅠ
    얘도 꼭 필수에유 …이것도 두번 더 추가해서 먹고싶었어요
    직원분도 계속 먹는거 확인해주시고 친절하세요 !
    최고최고 재방문 의사 100프로 💛💛💛🤤🤤🤤
    걍 너무 맛있어요 ~~~~~
    여튼 총 4개의 메뉴 중 저는 고등어봉초밥, 안키모가 시그니처라고 생긱합니다 ! 고등어봉초밥 넘 맛닜어요 담에 또 올게용 맛있어요!! 고등어 초밥이 특히 맛있었습니다 … 😍 재방문 의사 완전 있어요옹 고등어 봉초밥 처음먹어봤는데 비린내도 없고 너무 맛있었어요 ㅎㅎ 추천합니다 맛있는 음식과 분위기 기억에 오래남을듯 맛이 깔끔해요.
    스지 오뎅탕은 국물이 조금 심심한데
    저는 짜지 않아 먹을 수록 좋았어요.
    음악도 시끄럽지 않고
    직원 분들이 친절해요.
    2번째 방문 이예요 ~ 음식이 독특하고 매력있는 맛이에요 또 생각날거같아요 ㅎㅎ❤️ 너무 맛있게 먹고가요 개별 룸으로 주셔서 데이트하는데 좋았습니다 봉고등어초밥 맛있습니다 고등어 초밥이 너무 맛있어요🥹 진짜 최고최고!!! 또올거에요🥹♥️♥️ 고등어초밥 더 팔아주세용 ㅎㅎ 봉초밥 먹고 감동했어요. 인생 투톱입니다. 하나 더 시키고 싶은데 예약으로만 되는 게 슬프네요. 다른 메뉴들도 맛있습니다👍👍👍👍최고에요. 
    재방문 의사 10000프로에요🥰 재방문인데 음식도 맛있고 직원분들도 친절합니다~~!  이번에 기념일이라 방문했는데 너무 맛있게 잘 먹었습니다!
    봉초밥 상상이상이네요.... 정말 맛있었어요👍👍👍 진짜 너무 맛있어요.... 
    인생 숙성회를 만났습니다 
    꼭 재방문 하겠습니다 ! 고등어봉초밥 안키모 마끼 숙성 사시미 다 너무 맛있어요.  맛있고 분위기도 좋아요 :)
    친절하십니다💕 고등어봉초밥 많이 먹어봤는데 여기가 1등입니다!!
    감성도 좋고 최고최고 ㅠ 고등어 봉초밥 완전 맛있고
    분위기도 좋아서 완전 추천드립니다  ദ്ദി ˉ͈̀꒳ˉ͈́ ) 
    다른 메뉴도 짱맛!!  진짜 너무너무 맛있어요!! 고등어봉초밥도 맛있고, 모듬사시미 구성도 좋아요~! 기념일에 온 건데 정말 만족스럽습니다ㅎㅎ 다음에 또 오고 싶어요! 제가 너무너무 좋아하는 식당. 고등어봉초밥 안키모 최고맛집!!  분위기도 너무 좋고 직원분들이 다들 친절하셔서 기분좋게 맛있게 잘 먹었습니다~ 2주전에 오고 맛있어서 또 방문합니다‘ 안키모 진짜진짜진짜 너무 맛있어요!!
    크리미하고 ㅠㅠ 고등어봉초밥만 사람들이 노래부르는
    이유를 모르겠어여 ..
    안키모꼭드세요 두번드세요

    다음에 또와서 못먹어본거 꼭.. 또 먹어봐야지.. 꼭..🔥 고등어봉초밥 너무 맛있네요~~ 저녁시간에 예약해서 다시 오고 싶어요 음식이 진짜 다 맛있어요!! 회도 너무 신선하구 아귀 간도 진짜 듬뿍 올라가서 오이김밥이랑 넘 잘 어울리는거 같아요ㅠㅠ 도미마라도 먹었는데 마라맛도 너무 좋구 튀김이랑도 너무 잘 어울려요ㅠㅠㅎㅎㅎㅎ 분위기도 너무 좋아서 다음에 꼭 또 올거에요!😚😚 너무 맛있어요. 다음에 또 오고 싶어요. 특히 고등어봉초밥 진짜 맛있구, 분위기에 취할 정도로 최고에요! 와... 말이 필요가 없읍니다... 진짜 제가 그동안 알던 고등어는 고등어가 아니였군요... 명장은 명장인 이유가 있읍니다.... 퇴사 해서 기분 좋은 날, 맛있는 음식과 분위기 그리고 술까지 너무 좋네요🫶 두번째 방문인데 또 올거 같아요 음식이 모두 특이하고 맛있습니다. 너무 마음에 들어서 이번엔 가족과 함께 다시 방문했어요.  고등어 봉초밥은 안비리고
    새우마라는 저에겐 좀 매웠는데 담백(?)하니 맛있었어요
    고기는 부드럽고 안느끼하고 맛있어요!
    아늑하지만 지하여서 좀 아쉬웠어요
    강아지 동반시 가방에서 꺼내면 안된다고 합니다 참고하세요~= 고등어봉초밥이 먹고 싶어서 예약하고 방문했어요~^^ 사실 고등어회 비린내가 심해서 걱정했는데, 너무너무너무 맛있어여!!! 비린내때문에 해산물 못 먹는 남편도 맛있다고 했답니다!!^^ (거짓말 조금 보태서 참치느낌 나요 ㅋㅋㅋㅋ) 부타가꾸니는 입에서 살살 녹습니다. 제 입맛엔 조금 짜긴 하지만요;;; 반신반의로 방문했는데, 너무 맛있어서 즐거운 기억으로 남을 것 같아요😍

    서비스 참치마끼도 넘 맛있어요~ 고등어봉초밥 달인 가게답게 너무 맛있어요.. 처음 먹어본 고등어봉초밥이었는데 성공적입니다..♥
    부타가꾸니는 메쉬드포테이토랑 너무 잘어울렸어요!!
    장어솥밥도 장국과 함께나와서 넘넘 맛있게 먹었습니다ㅎ
    남자친구랑 또또또간집으로 계속 오려구여🤤 주문한 음식 둘다 맛있게 먹었어요 :) 사시미랑 전복 그리고 새우 누들? 시켰어요 다 맛있네요
    ㅠㅠ고등어봉초밥은 예약 못해서 못먹었는데 다음에 도전해보겠습니다! 너무 와보고싶었던 곳이였는데 음식이 생각보다 훨씬 담백하고 맛있었습니다! 웰컴푸드로 주신 토마토 리코타 치즈 핑거푸드도 너무 맛있었고 기억솥밥 강추드려요!! 예약하고 방문했어요 안했으면 큰일날뻔했네요 
    사람이 이렇게 많을 수가 ㅎㅎ
    그만한 이유가 있네요 음식 넘 맛있어요!
    여자친구가 셰프님들 요리천재아니냐며..ㅎㅎ
    잘먹었습니다^^ 오랜만에 왔는데 여전히 맛있어요! 룸은 처음 와보는데 시원하고 분리가 잘 되어있어서 좋었어요 여기 분위기도 넘 좋은데 음식도 다 마싯네요!!! 해방촌 숨은 맛집 느낌입니당 여기 새우 볶음면 마라 좋아하시면 완전 추천이요!!!! 고등어 봉초밥 맛있어요~~ 에피타이저도 주셔서 넘 좋았어요!! 고등어 봉초밥 아주 맛있어요~~~! 분위기도 좋고 룸에서 먹었는데 시원하고 프라이빗하게 저녁먹을수있습니다🙌🏻 고등어 봉초밥 넘 맛있습니댜
    부타가꾸니도! 밑에 매쉬드포테이토랑 너무 잘 어울려욤
    술잔 직접 고를 수 있는 것도 귀여운 포인트인 것 같아요  기념일을 맞아 예약하고 방문했는데, 기대 이상이었습니다. 지금까지 먹어본 고등어봉초밥 중 단연 최고였고, 앞으로 고등어봉초밥이 생각날 때면 무조건 이곳으로 와야겠다는 확신이 들었습니다. 

    다른 메뉴들도 하나같이 훌륭하여 어느 것 하나 아쉬움이 없었고, 덕분에 아주 기분 좋은 저녁 시간을 보낼 수 있었습니다. 다음번에는 꼭 다시 방문해 다른 메뉴들도 모두 맛보고 싶습니다. 자신 있게 주변 사람들에게도 적극 추천하고 싶은 맛집입니다. 화이트와인 죤마탱~ 봉초밥 진짜 무족권 먹어야댐..! 안쥬 다 맛있어요 ㅠ  또와도 또 좋습니다 
    재재재재방문 각 맛이 아주 깔끔 했어요 사시미와 봉초밥에 조합이 미쳤었습니다^^
    모든게 귿!!
    100일 기념일로 왔는데 정말 맛있구 여자친구가 엄청 좋아해요!! 느좋입니다!  친구들과 행복한  시간을 보낼 수 있는 곳이였습니다. 고등어 초밥 진짜 맛있고 다른 음식들도 다 맛있어서 흡입했습니다😉 친한 친구와 맛있는 음식 먹으며 얘기하고 싶어서 찾아온 곳인데, 너무 만족스러운 식사였습니다 ♥️  두번째 방문인데 음식이 정말 맛있어서 또 올게요:)  서울 여행와서 찾아보다가 왔는데 회가 너무 부드럽고 신선해서 술이 왕창 들어갑니다 ~~~ 분위기도 좋고 잘 먹다가 갑니다 음식이 너무 정갈하고 맛있어요. 다음번 모임은 가족들 데리고 오고싶습니다.  완전 맛있어요! 분위기도 좋아요
    둘이오기도 좋아서 해방촌 오면 꼭 방문하세요!
    예약하면 바로 들어올 수 있는 것도 좋아요 너무 맛있어요😋 지금까지 먹어본 고등어봉초밥 중 1등👍 숙성회 2인, 스지 어묵탕, 고등어 봉초밥 주문했는데 세 메뉴다 너무 맛있게 잘 먹었습니다! 특히 고등어 봉초밥이 김과 정말 잘어울리고 적당히 기름져 강추합니다! 고등어봉초밥 먹으러 두번째 오는데 고등어 봉초밥 진짜.. 너무 맛있어요 부타가꾸니도 진짜 맛있었습니다ㅠㅠㅜㅠ 
    가지라구도 맛있었고 여긴 몇번이고 더 올 곳이에요ㅠㅠ🥹❕  맛은 좋았는데 주방 옆 좌석에 앉는 게 불편했어요
    메뉴 고를 때 주방 입구에서 직원 분이 계속 기다리고 쳐다보고 계신다는 부담감이 들더라고요
    식당이 전체적으로 넓지 않아서 동선상 어쩔 수 없다는 건 이해하지만 많이 부담스러웠어요
    주방에 조그마한 커튼을 다는 것도 좋은 방법인 것 같습니다

    음식이 기대 이상으로 맛있었어요! 또 방문 하고 싶어요 ~.~ 분위기도 좋고 이야기하며 먹기 좋네요 고등어 봉초밥은 정말 비리지도 않고 맛있습니다! 처음 먹었는데 완전 추천이요 고등어 봉초밥이 너무 맛있어요..! 프라이빗 룸이라 조용하게 친구들이랑 잘 놀다 왔습니당 🥹 고등어봉초밥 먹으러 가끔 오는데
    여기가 먹어본 곳 중에서 제일 맛있어요 !
    친구랑 좋은 시간 보내고 갑니다 🥹🫶 두번째 방문인데 식당의 분위기도 너무 좋고, 음식도 다 맛있어요 특히 호소마끼가 고소하고 부드러운 느낌이 너무 좋고 부타가쿠니는 루꼴라랑 고기가 잘어울리고 메쉬드포테이토가 되게 크리미하고 좋은것같아요
    다음에도 또 오고싶어요! 음식 다 너무 맛있고 분위기도 너무 좋아요
    고등어봉초밥 진짜 맛있습니다!! 너무 맛있고 가격 가성비도 좋고 강추입니닷 음식이 모두 너무 맛있네요.
    특히 고등어봉초밥 추천합니다.
    콜키지 차지도 적당합니다.
    좋은 식당에서 즐겁게 잘 먹고 갑니다. 고등어봉초밥하고 안키모맛집이에요. 오랜만에 왔는데 여전히 맛있어요! 친한 언니가 추천해줘서 방문했는데 분위기도 너무 좋고 직원 분도 너무 친절하세용 
    특히 사시미 최고에요 >_<  고등어초밥??? 이렇게
    맛있는거에요 ????
    언빌리버블 !!! 다시 와도 맛있어용ㅎㅎㅎ직원분도 친절하시고 다들 예약해서 오시는 것 같아용 예약으로 오시는 더 추천드립니당!! 고등어봉초밥은 꼭 드세요!!🐟🖤 음식도 맛있고 분위기가 좋아요!! 고등어봉초밥 꼭 드세용 ㅎㅎ 고등어 초밥 진짜 너무 맛있어요!
    전반적으로 음식이 다 맛있어요 ㅎㅎ 고등어 봉초밥 달인이 운영하시는 집인줄 모르고 맛있어보여서 왔는데 충격받았어요 ,, 넘 맛나네요 숙성회도 짱 ! 봉초밥 무조건 꼭드세요 진짜 입안에서 파티가 열려요.. 꼭 드세요!!! 개추천이에요 사시미도 너무 회가 신선해요ㅠㅠ 이 두메뉴 추천합니당!!
    """
]

print(f"✅ {len(raw_documents)}개 긴 문서 준비 완료")
for i, doc in enumerate(raw_documents, 1):
    print(f"   매장 {i}: {len(doc)} 자")

✅ 1개 긴 문서 준비 완료
   매장 1: 56129 자


In [13]:
# ========================================
# 3-2. 청킹 기법 선택 및 docs 생성 (LangChain)
# ========================================

# LangChain의 RecursiveCharacterTextSplitter 사용
chunk_size = 300
chunk_overlap = 50
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function=len,
    separators=[
        "\n\n",  # 문단
        "\n",    # 줄바꿈
        ". ",    # 문장 (마침표 + 공백)
        ".",     # 마침표
        " ",     # 공백
        "",      # 문자
    ]
)

print("✅ LangChain RecursiveCharacterTextSplitter 초기화")
print(f"   • chunk_size: {chunk_size}")
print(f"   • chunk_overlap: {chunk_overlap}")
print(f"   • separators: 문단 → 문장 → 공백 → 문자 순서\n")

# 모든 문서를 청킹하여 docs 배열 생성
docs = []
doc_metadata = []  # 어느 원본 문서에서 왔는지 추적

for doc_idx, raw_doc in enumerate(raw_documents, 1):
    # LangChain으로 청킹
    chunks = text_splitter.split_text(raw_doc)
    
    print(f"매장 {doc_idx}: {len(raw_doc)}자 → {len(chunks)}개 청크로 분할")
    
    for chunk_idx, chunk in enumerate(chunks, 1):
        docs.append(chunk)
        doc_metadata.append({
            "shop_id": doc_idx,
            "chunk_id": chunk_idx,
            "original_length": len(raw_doc)
        })
        print(f"  청크 {chunk_idx}: {len(chunk)}자 - {chunk[:50]}...")

print(f"\n{'='*60}")
print(f"✅ 총 {len(docs)}개 청크 생성 완료 (원본 {len(raw_documents)}개 문서)")
print(f"{'='*60}")

✅ LangChain RecursiveCharacterTextSplitter 초기화
   • chunk_size: 300
   • chunk_overlap: 50
   • separators: 문단 → 문장 → 공백 → 문자 순서

매장 1: 56129자 → 284개 청크로 분할
  청크 1: 276자 - 심야식당 기억 덕분에 남자친구하구 근사한 데이트 하구 가요!! 고등어봉초밥 진짜 맛나요 ⸝...
  청크 2: 273자 - 신선하고 맛있게 먹고 갑니다! 
    다음에 또 방문할께요!!
    부산에서 왔어요 두...
  청크 3: 284자 - 플레이리스트도 종아서 다음에 또 올게요🪽
    안키모마끼랑 부타가꾸니 강추…🤎 음식도 분...
  청크 4: 45자 - 진짜 너무너무 맛있어용
    특히 고등어봉초밥!!!!!
    또먹고싶어요 ㅎㅎ...
  청크 5: 291자 - 다음엔 남자친구랑 가고 싶어용 음식도 맛있고 술잔도 고를 수 있는 기회가 너무 좋아요. 모...
  청크 6: 199자 - 너무 맛있었어요 ><
    회도 싱싱하구 퀄리티 좋고, 고등어 봉초밥 유명한 이유가 있었...
  청크 7: 264자 - 왜 미리 예약 받는지 이해가 가는 맛이였어요!!
    잘 먹다 갑니당 음식이 너무 훌륭하...
  청크 8: 206자 - 또올게요 맛있어요 안키모초밥과 숙성회. +매실하이볼 
    다음엔 고등어초밥먹으러 오겠습...
  청크 9: 256자 - 걱정한게 의미없을정도로 맛있어요!!!
    재방문할거에요 무조건!! 아직 메뉴가 다 안나...
  청크 10: 130자 - 예전에도 왔는데 이맘때 되면 항상 생각나요 고등어 봉초밥 무조건 드세요!!!!
    분위...
  청크 11: 294자 - 여자친구가 너무 좋아해요 😀 회사 사람들이랑 왔는데 너무너무 만족이에요
    제가 먹어본...
  청크 12: 114자 - 특히 고등어봉초밥은 필수로 꼭 드세요 !
    직원분들도 친절해서 기분이 더 좋았던 곳이...
  청크 13: 243자 - 고등

# 💡 청킹 기반 워크플로우 요약

**전체 흐름**:
```
1. 긴 원본 문서 (raw_documents)
   ↓
2. 청킹 처리 (LangChain RecursiveCharacterTextSplitter)
   - chunk_size=150, overlap=30
   - 구분자 우선순위: 문단 → 문장 → 공백 → 문자
   ↓
3. 청크 배열 생성 (docs + doc_metadata)
   ↓
4. Dense 임베딩 (SentenceTransformer)
   ↓
5. Sparse 인덱싱 (BM25)
   ↓
6. 벡터 DB 저장 (Qdrant)
   - Dense + Sparse 벡터
   - Metadata (shop_id, chunk_id 등)
   ↓
7. Hybrid Search (Dense + Sparse + RRF)
```

**LangChain 사용의 장점**:
- ✅ 검증된 청킹 알고리즘 (프로덕션 레벨)
- ✅ 자연스러운 문서 경계 유지 (문단/문장 우선 분할)
- ✅ 다양한 언어 지원 및 커스터마이징 가능
- ✅ 각 청크가 어떤 원본 문서에서 왔는지 추적 가능
- ✅ 검색 시 관련 청크만 반환하여 정확도 향상

# 청킹된 데이터 임베딩 & BM25 인덱싱

- 청킹된 docs를 Dense(임베딩) + Sparse(BM25)로 처리

In [14]:
# Dense 임베딩
dense_embeddings = sentenceTransformer.encode(docs)
print(f"✅ Dense 임베딩 생성: {dense_embeddings.shape}")

# BM25 인덱스 (전체 문서로 학습)
bm25 = BM25Okapi([doc.split() for doc in docs])
print(f"✅ BM25 인덱싱 완료")

✅ Dense 임베딩 생성: (284, 384)
✅ BM25 인덱싱 완료


# 테스트 데이터 저장 (Dense, Sparse)

In [15]:
# ========================================
# 5. 청킹된 데이터를 Qdrant에 저장
# ========================================

# Qdrant에 삽입 (metadata 포함)
points = []

for idx, (doc, dense, meta) in enumerate(zip(docs, dense_embeddings, doc_metadata)):
    sparse_scores = bm25.get_scores(doc.split())
    sparse_idx = np.where(sparse_scores > 0)[0]

    points.append(PointStruct(
        id=idx,
        vector={
            "dense": dense.tolist(),
            "sparse": SparseVector(
                indices=sparse_idx.tolist(),
                values=sparse_scores[sparse_idx].tolist(),
            )
        },
        payload={
            "text": doc,
            "shop_name": f"매장_{meta['shop_id']}",
            "shop_id": meta['shop_id'],
            "chunk_id": meta['chunk_id'],
            "original_length": meta['original_length']
        }
    ))

qdrantClient.upsert(collection_name=collection_name, points=points)

print(f"\n{'='*60}")
print(f"✅ {len(points)}개 청크 포인트 삽입 완료")
print(f"   • 원본 문서: {len(raw_documents)}개")
print(f"   • 총 청크: {len(docs)}개")
print(f"   • Dense 벡터 차원: {len(dense_embeddings[0])}")
print(f"{'='*60}")


✅ 284개 청크 포인트 삽입 완료
   • 원본 문서: 1개
   • 총 청크: 284개
   • Dense 벡터 차원: 384


# Hybrid Search (Dense + Sparse + RRF) 함수 정의

In [16]:
# ========================================
# 5-1. Hybrid Search 함수 (RRF) - 업데이트
# ========================================
def retrieve_similar_examples_hybrid(query, collection_name, top_k=2):
    # 1. query -> Dense 임베딩 생성
    query_dense_embedding = sentenceTransformer.encode(query)

    # 2. query -> Dense 검색
    dense_results = qdrantClient.query_points(
        collection_name=collection_name,
        query=query_dense_embedding.tolist(),
        using="dense",
        limit=10
    )

    # 3. query -> Sparse 벡터 생성 (BM25)
    query_tokens = query.split()
    query_sparse_scores = bm25.get_scores(query_tokens)
    query_sparse_idx = np.where(query_sparse_scores > 0)[0]

    # 4. query -> Sparse 검색
    sparse_results = qdrantClient.query_points(
        collection_name=collection_name,
        query=SparseVector(
            indices=query_sparse_idx.tolist(),
            values=query_sparse_scores[query_sparse_idx].tolist()
        ),
        using="sparse",
        limit=10
    )

    # 5. RRF 융합 (k=60은 표준 값)
    scores = defaultdict(float)
    k = 60

    # 6. Dense 점수 반영
    for rank, hit in enumerate(dense_results.points, 1):
        scores[hit.id] += 1 / (k + rank)

    # 7. Sparse 점수 반영
    for rank, hit in enumerate(sparse_results.points, 1):
        scores[hit.id] += 1 / (k + rank)

    # 8. 상위 top_k개 선택
    top_ids = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:top_k]

    # 9. 실제 데이터 가져오기 (metadata 포함)
    final_results = []
    for point_id, rrf_score in top_ids:
        point = qdrantClient.retrieve(collection_name=collection_name, ids=[point_id])[0]
        final_results.append({
            "shop_name": point.payload.get("shop_name", "Unknown"),
            "text": point.payload.get("text", ""),
            "shop_id": point.payload.get("shop_id"),
            "chunk_id": point.payload.get("chunk_id"),
            "score": rrf_score,
            "id": point_id
        })

    return final_results

print("✅ Hybrid Search 함수 업데이트 완료 (metadata 포함)")

✅ Hybrid Search 함수 업데이트 완료 (metadata 포함)


# Dense vs Sparse vs Hybrid Search (RFF) 비교

In [18]:
# ========================================
# 6. Dense vs Sparse vs Hybrid 비교
# ========================================

print(f"\n\n{'='*60}")
print("📊 Dense vs Sparse vs Hybrid 비교")
print(f"{'='*60}")

test_queries = [
    "미쉐린",         # 정확한 키워드
    "고급 레스토랑",    # 의미적 검색
    "스시",           # 키워드
    "분위기 좋은 곳"    # 의미적
]

for query in test_queries:
    # Dense only
    dense_only = qdrantClient.query_points(
        collection_name=collection_name,
        query=sentenceTransformer.encode(query).tolist(),
        using="dense",
        limit=3
    )

    print(f"\n🔹 Dense Only:")
        
    for hit in dense_only.points:
        print(f"  - {hit.payload['text'][:50]}... (Score: {hit.score:.4f})")

    # Sparse only
    query_tokens = query.split()
    sparse_scores = bm25.get_scores(query_tokens)
    sparse_idx = np.where(sparse_scores > 0)[0]

    sparse_only = qdrantClient.query_points(
        collection_name=collection_name,
        query=SparseVector(
            indices=sparse_idx.tolist(),
            values=sparse_scores[sparse_idx].tolist()
        ),
        using="sparse",
        limit=3
    )

    print(f"\n🔹 Sparse Only:")
    for hit in sparse_only.points:
        print(f"  - {hit.payload['text'][:50]}... (Score: {hit.score:.4f})")

    # Hybrid (DemRRF)
    hybrid_results = retrieve_similar_examples_hybrid(query, collection_name, top_k=3)

    print(f"\n🔹 Hybrid (Dense + Sparse + RRF):")
    for result in hybrid_results:
        print(f"  - {result['text'][:50]}... (Score: {result['score']:.4f})")

print("\n✅ 테스트 완료!")



📊 Dense vs Sparse vs Hybrid 비교

🔹 Dense Only:
  - .ㅎ... (Score: 0.8609)
  - 마싯움…👍🏻 비린맛 하나도 안 나구 꼭 드세여 너무 맛있어요!!!!! 고등어초밥 유명한 이... (Score: 0.4563)
  - 대박 맛도리! 고등어초밥 최고!! 비린내 하나도 안 나고 너무 맛있었어요 :) 사시미랑 아... (Score: 0.4520)

🔹 Sparse Only:

🔹 Hybrid (Dense + Sparse + RRF):
  - .ㅎ... (Score: 0.0164)
  - 마싯움…👍🏻 비린맛 하나도 안 나구 꼭 드세여 너무 맛있어요!!!!! 고등어초밥 유명한 이... (Score: 0.0161)
  - 대박 맛도리! 고등어초밥 최고!! 비린내 하나도 안 나고 너무 맛있었어요 :) 사시미랑 아... (Score: 0.0159)

🔹 Dense Only:
  - 생활의 달인에 나온집이더라고요...
    근데 예약하고도 몰랐네요😅... (Score: 0.7821)
  - 첫방문으로 생일파티를 하러 방문했는데 공간은 협소했지만 붐비는 느낌 없이 음식이 너무너무 ... (Score: 0.7272)
  - 여기 오시면 고등어봉초밥은 꼭 드세요..! 진짜 맛있습니다🤤
    친구랑 애인이랑 상관없... (Score: 0.6987)

🔹 Sparse Only:

🔹 Hybrid (Dense + Sparse + RRF):
  - 생활의 달인에 나온집이더라고요...
    근데 예약하고도 몰랐네요😅... (Score: 0.0164)
  - 첫방문으로 생일파티를 하러 방문했는데 공간은 협소했지만 붐비는 느낌 없이 음식이 너무너무 ... (Score: 0.0161)
  - 여기 오시면 고등어봉초밥은 꼭 드세요..! 진짜 맛있습니다🤤
    친구랑 애인이랑 상관없... (Score: 0.0159)

🔹 Dense Only:
  - 꼭 사시미 드시길 바랍니다... 입에서 녹아요....... (Score: 0.6540)
 